## Load required modules.

In [1]:
%matplotlib widget

from sklearn.decomposition import PCA, KernelPCA
import xlrd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import ipywidgets 
from ipywidgets import interact, IntSlider
import ipywidgets as widgets
from matplotlib import pyplot
from IPython.display import display, clear_output, Javascript
from matplotlib.colors import LinearSegmentedColormap
from sklearn.preprocessing import FunctionTransformer
from matplotlib.legend import DraggableLegend, Legend
import matplotlib

from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.pipeline import Pipeline
import gc
import psutil


import customtkinter as ctk
import tkinter as tk
from tkinter import ttk, scrolledtext
from tkinter import filedialog as fd
from tkinter import *

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from tkinter import IntVar
from tkinter import Menu
import matplotlib.gridspec as gridspec
from tkinter import filedialog as fd, simpledialog
from matplotlib.backend_bases import MouseButton
import re
from yellowbrick.cluster import KElbowVisualizer  
from PIL import Image, ImageTk
from tktooltip import ToolTip
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

from sklearn.cluster import AgglomerativeClustering, DBSCAN, SpectralClustering, MeanShift, AffinityPropagation, AgglomerativeClustering, Birch, OPTICS
from sklearn.mixture import GaussianMixture
import webbrowser
from customtkinter import CTkImage
from PIL import Image

import threading
import code
import sys
import io
from matplotlib.lines import Line2D

In [2]:
# window = ctk.CTk()

# #window
# window.title('MineralAI')
# window.geometry('1100x800')

# #widgets


# window.mainloop()

In [ ]:
class TerminalApp:
    def __init__(self, text_widget, main_app):
        # Use the passed text widget for terminal I/O
        self.terminal_text = text_widget
        self.main_app = main_app  # Reference to the main app

        # Define an example variable
        self.x = "hello"
        
        # Create a variable to store command history
        self.command_history = []
        self.history_index = -1

        # Create an interactive interpreter
        self.interpreter = code.InteractiveInterpreter(locals=self.get_scope())

        # Redirect stdout and stderr after interpreter is set
        sys.stdout = self
        sys.stderr = self

        # Insert initial prompt
        self.prompt()

        # Bind Enter and KeyPress events
        self.terminal_text.bind("<Return>", self.enter_command)
        self.terminal_text.bind("<KeyPress>", self.on_key_press)

    def get_scope(self):
        # Combine globals, instance attributes, and main app attributes
        scope = globals().copy()
        scope.update(self.__dict__)
        scope.update(self.main_app.__dict__)  # Include main app's attributes
        scope['self'] = self
        scope['main_app'] = self.main_app  # Explicitly add main_app to scope
        return scope

    def update_scope(self):
        # Ensure interpreter has the latest local variables
        self.interpreter.locals.update(self.get_scope())

    def __setattr__(self, name, value):
        super().__setattr__(name, value)
        if name not in ('interpreter', 'terminal_text', 'command_history', 'history_index', 'main_app') and 'interpreter' in self.__dict__:
            self.interpreter.locals[name] = value

    def __getattr__(self, name):
        if 'interpreter' in self.__dict__ and name in self.interpreter.locals:
            return self.interpreter.locals[name]
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")

    def write(self, message):
        # Use after to ensure this runs on the main thread
        self.terminal_text.after(0, self.terminal_text.insert, tk.END, message)
        self.terminal_text.after(0, self.terminal_text.see, tk.END)

    def flush(self):
        pass

    def prompt(self):
        self.write(">>")
        self.terminal_text.mark_set("insert", tk.END)
        self.terminal_text.see(tk.END)

    def enter_command(self, event):
        command = self.get_command()
        if command.strip():
            self.terminal_text.insert(tk.END, "\n")
            self.command_history.append(command)
            self.history_index = -1
            self.execute_command(command)
        return "break"

    def get_command(self):
        # Get the line where the command was entered
        line_start_index = self.terminal_text.index("insert linestart")
        line_end_index = self.terminal_text.index("insert lineend")
        command = self.terminal_text.get(line_start_index, line_end_index).strip()
        
        # Remove the prompt ('>>') from the command
        if command.startswith(">>"):
            command = command[2:]
        
        return command

    def execute_command(self, command):
        threading.Thread(target=self.run_command, args=(command,)).start()

    def run_command(self, command):
        try:
            self.update_scope()  # Ensure interpreter has the latest local variables
            self.interpreter.runsource(command)
        except Exception as e:
            self.write(str(e))
        self.prompt()

    def on_key_press(self, event):
        # Disallow editing previous text
        if self.terminal_text.compare("insert", "<", self.terminal_text.index(tk.END + "-1c linestart")):
            return "break"
        
        # Handle command history navigation
        if event.keysym == "Up":
            if self.command_history:
                self.history_index = max(0, self.history_index - 1)
                self.replace_command(self.command_history[self.history_index])
            return "break"
        elif event.keysym == "Down":
            if self.command_history:
                self.history_index = min(len(self.command_history) - 1, self.history_index + 1)
                self.replace_command(self.command_history[self.history_index])
            return "break"

    def replace_command(self, command):
        line_start_index = self.terminal_text.index("insert linestart")
        line_end_index = self.terminal_text.index("insert lineend")
        self.terminal_text.delete(line_start_index, line_end_index)
        self.terminal_text.insert(line_start_index, f">>{command}")








class main(ctk.CTk):   
    def __init__(self):
        super().__init__()
        self.title("MineralAI")
        ctk.set_appearance_mode("Light")
        style = ttk.Style()
        style.theme_use("default")
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        
        self.geometry(f"{screen_width}x{screen_height}")
        self.filtered_df = pd.DataFrame()
        self.create_widgets()

        #style = ttk.Style(self)
        self.tk_setPalette(background='white', foreground='black', selectBackground='SystemHighlight', activeForeground='black')
        
        # # Setting ttk widgets to match the light theme
        # style.theme_use('clam')
        
        # style.configure('TButton', background='white', foreground='black', borderwidth=1)
        # style.map('TButton', background=[('active', 'lightgray')])
    
        # style.configure('TLabel', background='white', foreground='black')
        # style.configure('TFrame', background='white')
        # style.configure('TEntry', fieldbackground='white', foreground='black')
        # style.configure('TCombobox', fieldbackground='white', background='white', foreground='black')

    
    def create_widgets(self):    
        # Create the main layout frames
        self.selection_frame = ctk.CTkFrame(self, height = 250)
        self.selection_frame.grid(row=1, column=0, rowspan=1, sticky="ns", pady=(5,0), padx=5)
        
        self.legend_frame = ctk.CTkFrame(self, width=225)
        self.legend_frame.grid(row=1, column=3, sticky="ns", padx=5, pady=(5,0))
    
        self.box_frame = ctk.CTkFrame(self, width=150)
        self.box_frame.grid(row=2, column=0, rowspan=2, sticky="nsew", padx=5, pady=5)
    
        self.box_frame_sub = ctk.CTkFrame(self, width=200)
        self.box_frame_sub.pack_propagate(False)
        self.box_frame_sub.grid(row=1, column=1, rowspan=3, sticky="ns", pady=5)
    
        self.bargraph_frame = ctk.CTkFrame(self)
        self.bargraph_frame.grid(row=1, column=2, rowspan=3, sticky="nsew", pady=5, padx=(5,0))
        
        self.output_frame = ctk.CTkFrame(self, width=225)
        self.output_frame.pack_propagate(False)
        self.output_frame.grid(row=2, column=3, sticky="nsew", padx=5, pady=5)
        
        self.grid_columnconfigure(2, weight=1)
        self.grid_rowconfigure(2, weight=1)

        self.output_text = scrolledtext.ScrolledText(self.output_frame, wrap="word", bd=0)
        self.output_text.pack(fill="both", expand=True)
    

        menubar = Menu(self)
        self.filemenu = Menu(menubar, tearoff=0)

        # Create file menu options
        self.filemenu.add_command(label="Open File", command=self.load_data)
        self.filemenu.add_command(label="New Tab", command=lambda: self.shared_container.create_tab("New Tab"))
        
        self.save_pc_menu = tk.Menu(self.filemenu, tearoff=0)
        
        # Create save PC options
        self.save_pc_menu.add_command(label="Save PC by sample excel", command=self.export_pc_by_sample)
        self.save_pc_menu.add_command(label="Save PC by element excel", command=self.export_pc_by_element)
        
        self.save_pc_menu.entryconfig("Save PC by sample excel", state=tk.DISABLED)
        self.save_pc_menu.entryconfig("Save PC by element excel", state=tk.DISABLED)
        
        self.filemenu.add_cascade(label="Save PC", menu=self.save_pc_menu)
        
        self.filemenu.add_separator()
        self.filemenu.add_command(label="Exit", command=self.destroy)
        menubar.add_cascade(label="File", menu=self.filemenu)
        
        helpmenu = Menu(menubar, tearoff=0)
        # Create help menu options
        helpmenu.add_command(label="Help Index", command=self.open_help_html)
        helpmenu.add_separator()
        helpmenu.add_command(label="Short-Cuts:", command=None)
        helpmenu.add_command(label="Deselect_all", command=None, accelerator="Cmd+d")
        helpmenu.add_separator()
        helpmenu.add_command(label="About...", command=None)
        menubar.add_cascade(label="Help", menu=helpmenu)
        self.config(menu=menubar)
        
        # self.terminal = TerminalApp(self.output_text, self)  # Pass self to TerminalApp


        
    def open_help_html(self):
        import os
        import urllib.parse
        
        # Open HTML help file
        html_path = '_build/html/index.html'
        absolute_path = os.path.abspath(html_path)
        file_url = urllib.parse.urljoin('file:', urllib.request.pathname2url(absolute_path))
        print(f"Opening HTML file at: {file_url}")
        
        try:
            if os.path.exists(absolute_path):
                webbrowser.open_new_tab(file_url)
                print("HTML file opened successfully.")
            else:
                print(f"File does not exist: {absolute_path}")
        except Exception as e:
            print(f"Error opening HTML file: {e}")

    def button_0(self):
        # Create sheet combobox and load button
        self.sheet_combobox = ctk.CTkComboBox(self.selection_frame, values=[], state="readonly")
        self.sheet_combobox.pack(padx=5, pady=5)
        self.sheet_combobox.set("Select a sheet")
        self.load_sheet_button = ctk.CTkButton(self.selection_frame, text="Load Sheet", command=self.load_selected_sheet)
        self.load_sheet_button.pack(pady=5)

    def create_lithobuttons(self):
        # Create lithology selection buttons
        ctk.CTkLabel(self.selection_frame, text="Select Lithology:").grid(row=0, column=0, columnspan=4, sticky="w", padx=5, pady=(5,0))
        self.lithology_listbox.grid(row=1, column=0, columnspan=4, sticky="we", padx=5, pady=(5,0))
        
        def deselect_all():
            self.lithology_listbox.selection_clear(0, tk.END)
            
        self.lithology_listbox.bind('<Command-d>', lambda event: deselect_all())
        self.create_buttons()

    def create_buttons(self):
        #initialize tab function
        self.shared_container = SharedContainer(self.bargraph_frame)
        
        # Create PCA and scaling options
        self.current_button = None
        
        self.scaler_combo = ctk.CTkComboBox(self.selection_frame, values=["Standard Scaler", "Logarithmic Scaler"])
        self.scaler_combo.grid(row=2, column=0, columnspan=4, sticky="we", padx=5, pady=(5,0))
        self.scaler_combo.set("Select PCA Scaler:")

        self.pca_type_combo = ctk.CTkComboBox(self.selection_frame, values=["PCA", "Kernel PCA"], command=self.kernelstat)
        self.pca_type_combo.grid(row=3, column=0, columnspan=4, sticky="we", padx=5, pady=(5,0))

        ctk.CTkLabel(self.selection_frame, text="Number of PCA Components:").grid(row=5, column=0, columnspan=4, sticky="w",  padx=5, pady=(5,0))
        self.slider = ctk.CTkSlider(self.selection_frame, from_=1, to=8, number_of_steps=7, command=self.on_slider_change)
        self.slider.set(6)
        self.slider.grid(row=6, column=0, columnspan=4, padx=10, pady=0)
        self.label = ctk.CTkLabel(self.selection_frame, text=f"Current value: {int(self.slider.get())}")
        self.label.grid(row=7, column=0, columnspan=4, padx=5, pady=0)

        self.apply_button = ctk.CTkButton(self.selection_frame, text="Apply", command= self.filter_dataframe)
        self.apply_button.grid(row=8, column=0, columnspan=4, padx=5, pady=0)
        
        self.kernel_combo = ctk.CTkComboBox(self.selection_frame, values=["linear", "poly", "rbf", "sigmoid", "cosine", "precomputed"])
        self.kernel_text = ctk.CTkLabel(self.selection_frame, text="Kernel:")

    def kernelstat(self, *args):
        # Show kernel options if Kernel PCA is selected
        pca_type = self.pca_type_combo.get()
        if pca_type == "Kernel PCA":
            self.kernel_combo.grid(row=4, column=1, columnspan=3, sticky="we", padx=5, pady=(5,0))
            self.kernel_text.grid(row=4, column=0, sticky="we", padx=5, pady=(5,0))
        else:
            self.kernel_combo.grid_forget()
            self.kernel_text.grid_forget()
            
    def on_slider_change(self, value):
        # Update label when slider changes
        self.label.configure(text=f"Current value: {int(value)}")

    def on_button_click(self, button, command):
        # Handle button click and change appearance
        if self.current_button is not None:
            self.current_button.configure(fg_color='transparent') #'#3B8ED0'
        else:
            pass
            
        button.configure(fg_color="grey")
        self.current_button = button
        command()

    def load_data(self):
        # Load data from selected file
        try:
            #self.file_path = 'Geochemistry Results-AGG reduced variables.xlsx'
            self.file_path = fd.askopenfilename()
            if not self.file_path:
                self.output_text.insert("end", "No file selected\n")
                return

            self.clear_all()
            
            self.button_0()
            self.sheet_names = pd.ExcelFile(self.file_path).sheet_names
            self.sheet_combobox.configure(values=self.sheet_names)
            self.output_text.insert("end", "File loaded successfully. Please select a sheet.\n")

        except:
            self.output_text.insert("end", f"Error loading file: \n")

    def load_selected_sheet(self):
        # Load selected sheet from the file
        try:
            selected_sheet = self.sheet_combobox.get()
            if selected_sheet not in self.sheet_names:
                self.output_text.insert("end", "Invalid sheet name\n")
                return
            self.sheet_name=selected_sheet
            self.df_0 = pd.read_excel(self.file_path, self.sheet_name)
            
            for widget in self.selection_frame.winfo_children():
                widget.destroy()
            self.process_sheet()
            
        except Exception as e:
            self.output_text.insert("end", f"Error loading sheet: {str(e)}\n")

    def process_sheet(self):
        # Process the sheet and populate lithology options
        if "Lithology" in self.df_0.columns:
            self.df_0['Lithology'] = self.df_0['Lithology'].apply(lambda x: x.strip().lower().title() if isinstance(x, str) and pd.notnull(x) else x)
            self.lithology_listbox = tk.Listbox(self.selection_frame, selectmode=tk.MULTIPLE)
            self.lithologies = self.df_0['Lithology'].unique()
            for lithology in self.lithologies:
                self.lithology_listbox.insert(tk.END, lithology)
            self.create_lithobuttons()

        else:
            self.output_text.insert("end", "Lithology column not found in dataframe\n")
            self.create_buttons()

    def clear_all(self):
        # Clear all widgets in frames
        for widget in self.bargraph_frame.winfo_children():
            widget.destroy()
        for widget in self.box_frame.winfo_children():
            widget.destroy()
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()
        # for widget in self.output_frame.winfo_children():
        #     widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
        for widget in self.selection_frame.winfo_children():
            widget.destroy()

    def clear(self):
        # Clear specific widgets in frames
        # for widget in self.bargraph_frame.winfo_children():
        #     widget.destroy()
        for widget in self.box_frame.winfo_children():
            widget.destroy()
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()
        self.output_text.delete("1.0", tk.END)
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

    def filter_dataframe(self):   
        #Filter dataframe based on selected lithology and update UI
        if hasattr(self, 'scaler_label') and self.scaler_label is not None:
            self.scaler_label.destroy()
            del self.scaler_label
            
        if self.scaler_combo.get() == "Select PCA Scaler:":
            self.scaler_label = ctk.CTkLabel(self.selection_frame, text="Select a Scaler", font=("Arial", 11))
            self.scaler_label.grid(row=10, column=0, columnspan=4, padx=5, pady=0)
            return
        else:
            # self.scaler_label.grid_forget()
            self.scaler_label = ctk.CTkLabel(self.selection_frame, text= f"Selected scaler: {self.scaler_combo.get()}", font=("Arial", 12))
            self.scaler_label.grid(row=9, column=0, columnspan=4, padx=5, pady=(0,5))
   
        self.clear()
        self.current_button = None
        if hasattr(self, 'lithologies_label') and self.lithologies_label is not None:
            self.lithologies_label.destroy()
            del self.lithologies_label

        # self.output_text = scrolledtext.ScrolledText(self.output_frame, wrap="word", bd=0)
        # self.output_text.pack(fill="both", expand=True)
        # TerminalApp(self.output_text)

        try:
            # Check selected lithologies and add check mark to them
            selected_indices = self.lithology_listbox.curselection()
            select_lithologies = [self.lithology_listbox.get(i) for i in selected_indices]
            selected_lithologies = [item.replace('✔️ ', '') for item in select_lithologies]
            
            self.filtered_df = self.df_0[self.df_0['Lithology'].isin(selected_lithologies)]
            
            item = None
            self.lithology_listbox.delete(0, tk.END)
            self.selected_items = {lithology: (lithology in selected_lithologies) for lithology in self.lithologies}
        
            for lithology, selected in self.selected_items.items():
                if selected:
                    item = "✔️ " + lithology
                else:
                    item = lithology
                self.lithology_listbox.insert(tk.END, item)
    
            def deselect_all():
                self.lithology_listbox.selection_clear(0, tk.END)
    
            self.lithology_listbox.bind('<Command-d>', lambda event: deselect_all())
            
            if not selected_lithologies:
                self.lithologies_label = ctk.CTkLabel(self.selection_frame, text="No lithology selected:(", font=("Arial", 11))
                self.lithologies_label.pack(side="top", padx=5, pady=(0,15))
            # else:
            #     lithologies_text = "\n".join(selected_lithologies)
            #     self.lithologies_label = ctk.CTkLabel(self.selection_frame, text=f"Selected Lithologies:\n{lithologies_text}", font=("Arial", 11))
            #     self.lithologies_label.pack(side="top", padx=5, pady=(0,15))
                                 
        except:
            self.filtered_df = self.df_0

        # Cleaning Data Frame to contain only the datas/elements
        filtered_columns = [col for col in self.filtered_df.columns if '_ppm' in col or '_pct' in col]

        df_filtered = self.filtered_df.replace('<', '', regex=True)
        df_filtered[filtered_columns] = df_filtered[filtered_columns].apply(pd.to_numeric, errors='coerce')
        self.cleaned_df = df_filtered.dropna(subset=filtered_columns)
        
        df_filtered = self.cleaned_df[filtered_columns]
        self.cleaned_df.columns = self.cleaned_df.columns.str.lower()

        self.df = df_filtered

        # Creating data frame with Rare Earth Elements
        self.df_c = self.df.copy()
        self.df_c.columns = self.df_c.columns.str.replace('_ppm', '').str.replace('_pct', '').str.replace('_Howell', '')
        
        LREE_elements = ['La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd']
        HREE_elements = ['Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu']
        
        LREE_elements_in_df = [element for element in LREE_elements if element in self.df_c.columns]
        HREE_elements_in_df = [element for element in HREE_elements if element in self.df_c.columns]
        
        if LREE_elements_in_df:
            self.df_c['LREE'] = self.df_c[LREE_elements_in_df].sum(axis=1)
        
        if HREE_elements_in_df:
            self.df_c['HREE'] = self.df_c[HREE_elements_in_df].sum(axis=1)
        
        if LREE_elements_in_df or HREE_elements_in_df:
            self.df_c['REE'] = self.df_c[LREE_elements_in_df + HREE_elements_in_df].sum(axis=1)

        # self.df_c['LREE'] = self.df_c[lree_elements].sum(axis=1)
        # self.df_c['HREE'] = self.df_c[hree_elements].sum(axis=1)
        
        # ree_elements = lree_elements + hree_elements
        # self.df_c['REE'] = self.df_c[ree_elements].sum(axis=1)

        # print(self.df_c)
        
        self.update_output()
        self.print_pca()

    def update_output(self):
        # Update output text with filtered and cleaned dataframe info
        # self.output_text.delete(1.0, "end")
        # self.output_text.insert("end", f"Filtered & Cleaned df:\n{self.df.head()}\n")
        
        self.output_text.insert("end", f"Filtered & Cleaned df:\n{self.df.shape}\n")
    
    def print_pca(self):
        # Perform PCA and update output
        try:
            self.pca_instance = PCA_class(self.df, self.scaler_combo, self.pca_type_combo, self.output_text, self.slider, self.kernel_combo)
            self.pca_instance.get_variance_ratio()
            self.output_text.insert("end", f"PCA performed successfully. Shape of transformed data: {self.pca_instance.x.shape}\n")
            #self.output_text.insert("end", f"PCA Components:\n{self.pca_instance.pca_df.head()}\n")
            self.color_function()
            self.shape_map()
            self.expand_buttons()
            self.loading_graph()   
            
        except ValueError as e:
            self.output_text.insert("end", f"{e}\n")

    def expand_buttons(self):
        # Create labels for PCA and Cluster Analysis sections
        self.pca_list = ctk.CTkLabel(self.box_frame, text="PCA Analysis Graphs")
        self.pca_list.grid(row=0, column=0, columnspan=6, sticky="w", pady=(10,0), padx=5)
        
        self.cluster_list = ctk.CTkLabel(self.box_frame, text="Cluster Analysis")
        self.cluster_list.grid(row=2, column=2, columnspan=3, sticky="w", pady=(10,0), padx=(25,0))

    def color_function(self):
        # Generate color mapping for lithologies
        try:
            self.dc = self.cleaned_df['lithology']
            self.dc = self.dc.to_frame()
            self.dc = self.dc.rename(columns={self.dc.columns[0]: 'Lithology'})
            
            # color_list = dc['Lithology'].unique()
            # default_colors = ["lime", "blue", "cyan", "deeppink", "pink", "black", "black", "black", "grey", "lightgrey"]
            # color_map = {}
            # # Populate the dictionary with symbols for each shape
            # for i, color in enumerate(color_list):
            #     if i < len(default_colors):
            #         color_map[color] = default_colors[i]
            self.color_map = {
                'Calcitic Marble': 'lime',
                'Dolomitic Marble': 'blue',
                'Di-Tr Dolomitic Marble': 'cyan',
                'Apatite Marble': 'deepskyblue',
                'Siliceous Calcitic Marble': 'steelblue',
                'Carbonatite': 'deeppink',
                'Carbonatite-Like': 'pink',
            
                'Syenite': 'black',
                'Altered Syenite': 'black',
                'Intrusion': 'black',
                'Skarn': 'sienna',
                'Syenite-Like': 'black',
                'Impure Siliciclastic': 'dimgrey',
                'Pure Siliciclastic': 'lightgrey',
                'Siliciclastic': 'grey'
                
            }
            self.dc['Color'] = self.dc['Lithology'].map(self.color_map)
            #self.output_text.insert("end", self.dc)
        except Exception as e:
            self.color_map = {}
            self.dc = pd.DataFrame()
            self.output_text.insert("end", f"Error generating lithology-color map: {e}\n")     
        
    def shape_map(self):
        # Generate shape mapping for rock units
        try:
            self.cleaned_df['rock unit'] = self.cleaned_df['rock unit'].apply(lambda x: x.strip().lower().title() if isinstance(x, str) and pd.notnull(x) else x)
            shape_list = self.cleaned_df['rock unit'].unique()
            # print(shape_list)
            
            self.ds = self.cleaned_df['rock unit']
            self.ds = self.ds.str.strip().to_frame()
            self.ds = self.ds.rename(columns={self.ds.columns[0]: 'Shape'})
            
            #shape_list = ds['Shape'].unique()
            # default_symbols = ["^", "*", "o", "s", "D"]
            # color_map1 = {}
            # # Populate the dictionary with symbols for each shape
            # for i, shape in enumerate(shape_list):
            #     if i < len(default_symbols):
            #         color_map1[shape] = default_symbols[i]
            
            self.color_map1 = {
                'Marble Units': "^",
                'Altered Intrusion': "*",
                'Siliciclastic': "o",
                'Intrusion': "s",
                'Anomalous Rock': "D",
            }
    
            self.ds["Shapes"] = self.ds["Shape"].map(self.color_map1)
            #self.output_text.insert("end", self.ds)Shape

        except Exception as e:
            self.color_map1 = {}
            self.ds = pd.DataFrame()
            self.output_text.insert("end", f"Error generating rock unit-shape map: {e}\n") 
            self.output_text.insert("end", f"Column Rock Unit does not exist, or check error in name?\n") 

    def loading(self):
        # Create loadings
        self.pca = self.pca_instance.pca

        if isinstance(self.pca, PCA):
            # print(f"pca: {self.pca}")
            components = self.pca.components_
            # print(f"components: {components}")
            self.loadings = pd.DataFrame(
                components.T,
                columns=['PC' + str(i) for i in range(1, components.shape[0] + 1)],
                index=self.df.columns
            )
            # self.loadings.index = self.loadings.index.str.replace('_ppm', '').str.replace('_pct', '').str.replace('(int)', '').str.replace('_Howell', '')
        elif isinstance(self.pca, KernelPCA):
            # print(f"pca: {self.pca}")
            components = self.pca_instance.kx
            # print(f"components: {components}")
            components_df = pd.DataFrame(components)
            components_df.reset_index(drop=True, inplace=True)
            components = components_df.to_numpy()
            self.loadings = pd.DataFrame(
                components,
                columns=['PC' + str(i) for i in range(1, components.shape[1] + 1)],
                index=self.df.columns
            # self.loadings.index = self.loadings.index.str.replace('_ppm', '').str.replace('_pct', '').str.replace('(int)', '').str.replace('_Howell', '')
            )
        else:
            raise ValueError("self.pca must be an instance of PCA or KernelPCA")
    
    def selection(self):
        # Create a combo box for selecting clustering methods
        self.cluster_combo = ctk.CTkComboBox(
            self.box_frame, values=["K-mean", "Hierarchical", "DBSCAN", "Mean Shift", "Spectral", "GMM", "Affinity Propagation", "BIRCH"], width=100, height=20, command=self.update_cluster)
        # self.cluster_combo.bind("<<ComboboxSelected>>", self.on_combobox_selected)
        self.cluster_combo.grid(row=2, column=0, columnspan=3, sticky="w", pady=(10, 0), padx=5)
        self.cluster = self.cluster_combo.get()
        self.twod_instance = Cluster2DPlotClass(self.shared_container, self.cluster, self.df_c, self.cleaned_df, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.color_map, self.color_map1, self.on_button_click, self.legend_frame)
        self.threed_instance = Cluster3DPlotClass(self.shared_container, self.cluster, self.df_c, self.cleaned_df, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.color_map, self.color_map1, self.on_button_click, self.legend_frame)


    def update_cluster(self, *arg):
        self.cluster = self.cluster_combo.get()
        # rerun plot_2d_cluster_sub to refresh options in box_frame_sub
        print("clusterchanged1")
        for widget in self.box_frame_sub.winfo_children():
            # print(widget.winfo_name())
            if widget.winfo_name() == "size_combo1":
                self.twod_instance.plot_2d_cluster_sub(self.cluster)
                print("2d")
            elif widget.winfo_name() == "size_combo":
                self.threed_instance.plot_3d_cluster_sub(self.cluster)  
                print("3d")
            else:
                print("pass")

    def loading_graph(self):
        # preparing data
        self.loading()
        self.pca_df_scaled = self.pca_instance.pca_df_scaled
        self.pca_df_scaled.reset_index(drop=True, inplace=True)
        #self.loadings.reset_index(drop=True, inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        self.dc.reset_index(drop=True, inplace=True)
        self.ds.reset_index(drop=True, inplace=True)
        self.cleaned_df.reset_index(drop=True, inplace=True)
        self.df_c.reset_index(drop=True, inplace=True)

        # Load graph with PCA and clustering
        self.selection()
        loading_class(self.loadings, self.shared_container, self.box_frame, self.box_frame_sub, self.on_button_click, self.apply_button)
        loading_cluster(self.shared_container, self.cluster, self.loadings, self.box_frame, self.box_frame_sub, self.on_button_click, self.apply_button, self.legend_frame)
        threed_class(self.shared_container, self.pca_df_scaled, self.df, self.cleaned_df, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.color_map, self.color_map1, self.on_button_click, self.apply_button, self.legend_frame, self.loadings)
        twod_class(self.shared_container, self.pca_df_scaled, self.df, self.cleaned_df, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.color_map, self.color_map1, self.on_button_click, self.apply_button, self.legend_frame, self.loadings)

        
        if "sample id" in self.cleaned_df.columns:
            sample_class(self.shared_container, self.pca_df_scaled, self.df, self.cleaned_df, self.box_frame, self.box_frame_sub, self.on_button_click, self.apply_button)
            sample_cluster(self.shared_container, self.cluster, self.pca_df_scaled, self.df, self.cleaned_df, self.box_frame, self.box_frame_sub, self.dc, self.ds, self.on_button_click, self.apply_button, self.legend_frame)
            drill_class(self.shared_container, self.pca_df_scaled, self.cleaned_df, self.df, self.box_frame, self.box_frame_sub, self.on_button_click, self.apply_button)
        else:
            print("Column Sample ID not in data frame, sample bar graph, sample cluster, and drill hole plot not avaliable")
            
            # pil_image = Image.open("images program/id.png")
            # # resized_image = pil_image.resize((32, 32), Image.LANCZOS)
            # self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))
    
            # self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text = "", width=20)
            # self.image_button.grid(row=3, column=1, sticky="w", pady=0, padx=5)

            
        self.save_pc_menu.entryconfig("Save PC by element excel", state=tk.NORMAL)
        if "sample id" in self.cleaned_df.columns:
            self.save_pc_menu.entryconfig("Save PC by sample excel", state=tk.NORMAL)
        
    def export_pc_by_sample(self):
        # Export PCA results by sample
        self.pca_df_scaled = self.pca_instance.pca_df_scaled
        self.pca_df_scaled.shape
        file_name = fd.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
        if not file_name:
            return

        with pd.ExcelWriter(file_name, engine='openpyxl') as excel_writer:
            self.cleaned_df["sample id"].to_excel(excel_writer, sheet_name='Sheet1', index=False, header=True)
            # self.cleaned_df["sample id"].shape
            self.pca_df_scaled.to_excel(excel_writer, sheet_name='Sheet1', startcol=1, index=False)
    
    def export_pc_by_element(self):
        # Export PCA loadings by element
        self.loadings = self.loading_instance.loadings
        file_name = fd.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
        if not file_name:
            return
        
        with pd.ExcelWriter(file_name, engine='openpyxl') as excel_writer:
            columns_df = pd.DataFrame(self.df.columns, columns=['Column Names'])
            columns_df.to_excel(excel_writer, sheet_name='Sheet1', index=False, header=True)
            self.loadings.to_excel(excel_writer, sheet_name='Sheet1', startcol=1, index=False)







class SharedContainer:
    def __init__(self, parent):
        # Initialize the tab container and content container
        self.tab_container = ctk.CTkFrame(parent)
        self.tab_container.pack(fill='x')
        self.content_container = ctk.CTkFrame(parent)
        self.content_container.pack(fill='both', expand=True)
        self.tabs = []
        self.current_tab = None

    def create_tab(self, title):
        # Create a new tab with a title
        tab_frame = tk.Frame(self.tab_container, bd=1, relief=tk.RAISED)
        tab_label = ctk.CTkLabel(tab_frame, text=title)
        tab_label.bind("<Double-1>", lambda event: self.edit_tab_label(tab_label))
        custom_font = ctk.CTkFont(size=16)
        close_button = ctk.CTkButton(tab_frame, text="x", width=25, height=25, border_width=0, font=custom_font, hover_color="darkgrey", text_color="black", command=lambda: self.close_tab(tab_frame, content_frame))

        tab_label.pack(side=tk.LEFT, padx=3, pady=3)
        close_button.pack(side=tk.RIGHT, padx=3, pady=3)
        tab_frame.pack(side=tk.LEFT, padx=2, pady=2)

        content_frame = ctk.CTkFrame(self.content_container)
        content_frame.pack(fill='both', expand=True)

        tab_frame.bind("<Button-1>", lambda event: self.select_tab(tab_frame, content_frame))
        tab_label.bind("<Button-1>", lambda event: self.select_tab(tab_frame, content_frame))
        close_button.bind("<Button-1>", lambda event: self.select_tab(tab_frame, content_frame))

        self.tabs.append((tab_frame, content_frame))
        self.select_tab(tab_frame, content_frame)

    def select_tab(self, tab_frame, content_frame):
        # Highlight the selected tab and show its content
        if self.current_tab:
            self.current_tab[0].config(bg='SystemButtonFace')
            for widget in self.current_tab[0].winfo_children():
                widget.configure(fg_color='SystemButtonFace')

        tab_frame.config(bg='lightgrey')
        for widget in tab_frame.winfo_children():
            widget.configure(fg_color='lightgrey')

        for frame in self.content_container.winfo_children():
            frame.pack_forget()

        content_frame.pack(fill='both', expand=True)
        self.current_tab = (tab_frame, content_frame)

    def close_tab(self, tab_frame, content_frame):
        # Close the selected tab and select the previous tab if available
        if self.current_tab == (tab_frame, content_frame):
            self.current_tab = None
        tab_frame.destroy()
        content_frame.destroy()
        self.tabs.remove((tab_frame, content_frame))
        if self.tabs:
            self.select_tab(*self.tabs[-1])

    def edit_tab_label(self, tab_label):
        # Enable editing of the tab label
        current_text = tab_label.cget("text")
        tab_frame = tab_label.master

        entry = tk.Entry(tab_frame, width=10)
        entry.insert(0, current_text)
        entry.pack(side=tk.LEFT, padx=3, pady=3)
        entry.bind("<Return>", lambda event: self.update_tab_label(tab_label, entry))
        entry.bind("<FocusOut>", lambda event: self.update_tab_label(tab_label, entry))
        entry.focus()

        tab_label.pack_forget()

    def update_tab_label(self, tab_label, entry):
        # Update the tab label with new text
        new_text = entry.get()
        tab_label.configure(text=new_text)
        entry.destroy()
        tab_label.pack(side=tk.LEFT, padx=3, pady=3)



# class kmean:
#     def __init__(self, box_frame, frame):
#         # Initialize the K-mean clustering selection interface
#         self.box_frame = box_frame
#         self.cluster = 'K-mean'
#         self.selection()
#         self.frame = frame

#     def selection(self):
#         # Create a combo box for selecting clustering methods
#         self.scaler_combo = ctk.CTkComboBox(
#             self.box_frame, values=["K-mean", "Hierarchical", "DBSCAN", "Mean Shift", "Spectral", "GMM", "Affinity Propagation", "BIRCH"], width=100, height=20, command=self.result)
#         self.scaler_combo.bind("<<ComboboxSelected>>", self.on_combobox_selected)
#         self.scaler_combo.grid(row=2, column=0, columnspan=3, sticky="w", pady=(10, 0), padx=5)

#         # self.scaler_combo.bind("<<ClusterChanged>>", self.on_cluster_changed)

#     def result(self, *args):
#         # Update the selected clustering method and generate a custom event
#         self.cluster = self.scaler_combo.get()
#         #self.scaler_combo.event_generate("<<ClusterChanged>>")
#         print("clusterchanged1")
#         for widget in self.frame.winfo_children():
#             print(widget.winfo_name())
#             if widget.winfo_name() == "size_combo1":
#                 Cluster2DPlotClass.plot_2d_cluster_sub()
#                 print("2d")
#             elif widget.winfo_name() == "size_combo":
#                 Cluster3DPlotClass.plot_3d_cluster_sub()  
#                 print("3d")
#             else:
#                 print("pass")

#     def get_cluster(self):
#         # Return the selected clustering method
#         return self.cluster

#     def on_cluster_changed(self, event):
#         print("Cluster changed!") # This should now be printed

#     def on_combobox_selected(self, event):
#         # This method is a placeholder if needed for additional actions
#         pass






class PCA_class:
    def __init__(self, df, scaler_combo, pca_type_combo, output_text, slider, kernel_combo):
        # Initialize the PCA class with necessary parameters and perform PCA
        self.df = df
        self.scaler_combo = scaler_combo
        self.pca_type_combo = pca_type_combo
        self.kernel_combo = kernel_combo
        self.output_text = output_text
        self.slider = slider
        self.perform_pca()
        self.bargraph_frame = None

    def perform_pca(self):
        # Perform scaling and PCA based on selected options
        if self.scaler_combo.get() == "Standard Scaler":
            scaling = StandardScaler()
        elif self.scaler_combo.get() == "Logarithmic Scaler":
            scaling = FunctionTransformer(np.log10, validate=True)
        try:
            scaling.fit(self.df)
            Scaled_data = scaling.transform(self.df)
            Scaled_df = pd.DataFrame(Scaled_data)
            
            pca_type = self.pca_type_combo.get()
            kernel_type = self.kernel_combo.get()
            if pca_type == "PCA":
                self.pca = PCA(n_components=int(self.slider.get()))
                self.kpca = PCA(n_components=int(self.slider.get()))
            elif pca_type == "Kernel PCA":
                self.pca = KernelPCA(n_components=int(self.slider.get()), kernel=kernel_type)
                self.kpca = KernelPCA(n_components=int(self.slider.get()), kernel=kernel_type)
            else:
                raise ValueError("Invalid PCA type selected")
                
            self.pca.fit(Scaled_data)
            self.x = self.pca.transform(Scaled_data)
    
            Scaled_data_transposed = Scaled_data.T
            self.kpca.fit(Scaled_data_transposed)
            self.kx = self.kpca.transform(Scaled_data_transposed)
    
            self.create_pca_df()
         
        except Exception as e:
            raise ValueError(f'Not enough components for PCA. Please select more!{e}') from e

    def create_pca_df(self):  
        # Create a DataFrame for the PCA results
        if isinstance(self.pca, PCA):
            num_components = self.pca.n_components_
        elif isinstance(self.pca, KernelPCA):
            num_components = self.pca.n_components
        
        self.pca_df = pd.DataFrame(
            data=self.x, 
            columns=['PC'+str(i) for i in range(1, num_components+1)]
        )
        self.scale_pca()
        
    def scale_pca(self):
        # Scale the PCA results
        self.pca_df_scaled = self.pca_df.copy()
        scaler_df = self.pca_df[self.pca_df.columns]
        scaler = 1 / (scaler_df.max() - scaler_df.min())
        for index in scaler.index:
            self.pca_df_scaled[index] *= scaler[index]
    
    def get_variance_ratio(self):
        # Output the explained variance ratio for each principal component
        if isinstance(self.pca, PCA) and hasattr(self.pca, 'explained_variance_ratio_'):
            explained_variance_ratio = self.pca.explained_variance_ratio_
            for i, variance in enumerate(explained_variance_ratio, start=1):
                self.output_text.insert("end", f'Principal Component {i}: \n{variance:.2%} of variance\n')
        else:
            self.output_text.insert("end", "Kernel PCA does not provide explained variance ratio.\n")






class loading_class:
    def __init__(self, loadings, shared_container, box_frame, box_frame_sub, on_button_click, apply_button):
        # Initialize the loading class with necessary parameters
        self.apply_button = apply_button
        self.shared_container = shared_container
        self.on_button_click = on_button_click
        self.loadings = loadings

        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.var = IntVar()
        self.group_list = []

        # Call method to create the PC graph button
        self.Graph_PC()

    def Graph_PC(self):
        # Create button for displaying PC bar graph
        pil_image = Image.open("images program/element-svgrepo-com.png")
        # resized_image = pil_image.resize((32, 32), Image.LANCZOS)
        self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))

        # Debug prints for loaded images
        # print(pil_image)
        # print(self.icon_image)

        # Create and place image button
        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text="", width=20)
        self.image_button.grid(row=1, column=0, sticky="w", pady=0, padx=5)

        # Bind button click event to PC graph function
        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, self.Graph_PC_sub))

        # Add tooltip to the button
        ToolTip(self.image_button, msg="PC Bar Graph by Elements")

    def Graph_PC_sub(self):
        # Create elements and groups selection UI
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        # Create and pack checkbox for sorting loadings
        self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Sort Loadings", variable=self.var)
        self.checkbox.pack(side="top", anchor="w", padx=5, pady=3)

        # Create and pack listbox for element selection
        self.elements_listbox = tk.Listbox(self.box_frame_sub, selectmode=tk.MULTIPLE)
        for element in self.loadings.index:
            self.elements_listbox.insert(tk.END, element)
        self.elements_listbox.pack(side="top", padx=5, pady=5)

        # Function to deselect all elements
        def deselect_all():
            self.elements_listbox.selection_clear(0, tk.END)

        # Bind deselect function to the listbox
        self.elements_listbox.bind('<Command-d>', lambda event: deselect_all())

        # Create and pack button to add selected elements as a group
        self.add_group_button = ctk.CTkButton(self.box_frame_sub, text="Add Group", command=self.add_group)
        self.add_group_button.pack(side="top", padx=5, pady=5)

        # Create and pack listbox to display added groups
        self.group_listbox = tk.Listbox(self.box_frame_sub, selectmode=tk.MULTIPLE)
        self.group_listbox.pack(side="top", padx=5, pady=5)

        # Create and pack apply button
        self.apply_Graph_PC = ctk.CTkButton(self.box_frame_sub, text="Apply", command=self.update_plots)
        self.apply_Graph_PC.pack(side="top", padx=5, pady=5)

        # Create and pack button to clear all groups
        self.clear_groups_button = ctk.CTkButton(self.box_frame_sub, text="Clear Groups", command=self.clear_groups)
        self.clear_groups_button.pack(side="top", padx=5, pady=5)

    def clear_groups(self):
        # Clear all groups
        self.group_list.clear()
        self.group_listbox.delete(0, tk.END)

    def add_group(self):
        # Add selected elements as a group
        selected_elements = [self.elements_listbox.get(i) for i in self.elements_listbox.curselection()]
        self.group_list.append(selected_elements)
        self.group_listbox.insert(tk.END, f"Group {len(self.group_list)}: {', '.join(selected_elements)}")

    def normalize(self, values, vmin=None, vmax=None):
        # Normalize values for color scaling
        vmin = vmin if vmin is not None else np.min(values)
        vmax = vmax if vmax is not None else np.max(values)
        norm_values = (values - vmin) / (vmax - vmin)
        return norm_values

    def plot_pc_barchart(self, data, pc_name, ax, sort, group_indices):
        # Plot the PC bar chart
        if sort:
            data = data.sort_values(ascending=False)
        norm_values = self.normalize(data, vmin=-max(abs(data)), vmax=max(abs(data)))
        bar_colors = plt.cm.coolwarm(norm_values)
        data.plot(kind='barh', color=bar_colors, ax=ax, fontsize="x-small")
        ax.set_title(pc_name)
        ax.grid(True, linestyle='--', linewidth=0.5)
        ax.set_ylabel(' ')

        # Draw horizontal lines to separate groups
        for idx in group_indices:
            ax.axhline(y=idx - 0.5, color='black', linewidth=1)

        # Update y-axis labels
        labels = ax.get_yticklabels()
        new_labels = [label.get_text().replace('_ppm', '')
                                      .replace('_pct', '')
                                      .replace('_Howell', '') for label in labels]
        ax.set_yticklabels(new_labels)

    def update_plots(self):
        # Update plots with selected groups and options
        sort = self.var.get() == 1
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("Element BarGraph")

        # Close all existing plots and clear widgets
        plt.close('all')
        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()

        # Create a new figure and axes for the plots
        fig, axes = plt.subplots(ncols=len(self.loadings.columns), figsize=(15, 8))
        if len(self.loadings.columns) == 1:
            axes = [axes]

        # Iterate over each principal component
        for i in range(len(self.loadings.columns)):
            combined_data = pd.Series(dtype='float64')
            group_indices = []
            current_index = 0
            selected_indices = self.group_listbox.curselection()
            
            # Combine data for selected groups
            for index in selected_indices:
                group = self.group_list[index]
                group_data = self.loadings.loc[group, f'PC{i+1}']
                if sort:
                    group_data = group_data.sort_values(ascending=False)
                combined_data = pd.concat([combined_data, group_data])
                current_index += len(group_data)
                group_indices.append(current_index)

            # Plot the combined data
            if not combined_data.empty:
                self.plot_pc_barchart(combined_data, f'PC{i+1}', axes[i], sort=False, group_indices=group_indices)

        # Adjust layout and add colorbar
        plt.tight_layout()
        fig_c = plt.figure(figsize=(12, 1))
        min_val = self.loadings.min().min()
        max_val = self.loadings.max().max()
        norm = plt.Normalize(vmin=min_val, vmax=max_val)
        cmap = plt.cm.coolwarm
        cbar = fig_c.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes, orientation='horizontal', fraction=0.02)
        
        # Add the plot to the tkinter canvas
        canvas = FigureCanvasTkAgg(fig, master=content_frame)
        canvas.draw()

        # Add navigation toolbar to the canvas
        toolbar = NavigationToolbar2Tk(canvas, content_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)







class loading_cluster:
    def __init__(self, shared_container, cluster, loadings, box_frame, box_frame_sub, on_button_click, apply_button, legend_frame):
        # Initialize the loading cluster class with necessary parameters
        self.loadings = loadings
        self.shared_container = shared_container
        self.cluster_result = cluster
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.legend_frame = legend_frame
        self.on_button_click = on_button_click
        self.apply_button = apply_button
        self.cluster_sort_var = IntVar()
        self.graph_data_df = None

        self.plot_cluster()

    def plot_cluster(self):
        # Create button for displaying cluster bar graph
        pil_image = Image.open("images program/ele.png")
        # resized_image = pil_image.resize((32, 32), Image.LANCZOS)
        self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))
        
        # Debug prints for loaded images
        print(pil_image)
        print(self.icon_image)
        
        # Create and place image button
        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text="", width=20)
        self.image_button.grid(row=3, column=0, sticky="w", pady=0, padx=5)

        # Bind button click event to cluster function
        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, self.plot_cluster_sub))

        ToolTip(self.image_button, msg="PC Cluster Bar Graph by Elements")

    def plot_cluster_sub(self):
        # Create elements and clusters selection UI
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        # Create and pack checkbox for sorting clusters
        self.cluster_sort_checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Sort Clusters Together", variable=self.cluster_sort_var)
        self.cluster_sort_checkbox.pack(side="top", anchor="w", padx=5, pady=(3, 0))

        # Create and pack slider for number of clusters
        ctk.CTkLabel(self.box_frame_sub, text="Number of Clusters:").pack(side="top", padx=5, pady=(5, 0))
        self.k_slider = tk.Scale(self.box_frame_sub, from_=2, to=10, orient=tk.HORIZONTAL)
        self.k_slider.pack(side="top", padx=5, pady=(5, 0))

        # Create and pack apply button
        self.apply_plot_3d_cluster = ctk.CTkButton(self.box_frame_sub, text="Apply", command=self.update_plots)
        self.apply_plot_3d_cluster.pack(side="top", padx=5, pady=5)

        # Create and pack save button
        self.save_button = ctk.CTkButton(self.box_frame_sub, text="Save Clusters to Excel", command=self.save_loadings_to_excel)
        self.save_button.pack(side="bottom", padx=5, pady=5)

    def normalize(self, values, vmin=None, vmax=None):
        # Normalize values for color scaling
        vmin = vmin if vmin is not None else np.min(values)
        vmax = vmax if vmin is not None else np.max(values)
        norm_values = (values - vmin) / (vmax - vmin)
        return norm_values

    def plot_pca_barchart(self, data, entry_name, ax, cluster_colors, sorted_labels):
        # Plot the PCA bar chart with clusters
        cluster_palette = plt.get_cmap('tab20', len(np.unique(cluster_colors)))
        colors = cluster_palette(cluster_colors)
        data.plot(kind='barh', color=colors, ax=ax, fontsize="xx-small")
        ax.set_title(entry_name)
        ax.grid(True, linestyle='--', linewidth=0.5)
        ax.set_yticks(range(len(data)))
        ax.set_yticklabels(sorted_labels, fontsize="xx-small") 
        ax.set_ylabel('')

        # Update y-axis labels
        labels = ax.get_yticklabels()
        new_labels = [label.get_text().replace('_ppm', '')
                                      .replace('_pct', '')
                                      .replace('_Howell', '') for label in labels]
        ax.set_yticklabels(new_labels)

    def estimate_eps(self, k):
        # Estimate the optimal eps value for DBSCAN clustering
        from sklearn.neighbors import NearestNeighbors
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

        # Compute the k-nearest neighbors
        nearest_neighbors = NearestNeighbors(n_neighbors=k)
        nearest_neighbors.fit(self.loadings)
        distances, indices = nearest_neighbors.kneighbors(self.loadings)

        # Sort the distances (distances are ordered for each point)
        distances = np.sort(distances, axis=0)
        distances = distances[:, 1]  # Taking the distance to the k-th nearest neighbor

        # Plot the k-distance graph
        fig, ax = plt.subplots(figsize=(3, 2))
        ax.plot(distances)
        ax.set_title('k-Distance Graph')
        ax.set_xlabel('Points sorted by distance')
        ax.set_ylabel(f'{k}th Nearest Neighbor Distance')

        # Add the plot to the tkinter canvas
        self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
        self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        plt.show()

        elbow_point_index = 20
        eps_value = distances[elbow_point_index]
        return eps_value

    def update_plots(self):
        # Update plots with selected clustering method and options
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("Cluster Bar Graph by Elements")

        # Close all existing plots and clear widgets
        plt.close('all')
        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

        k = self.k_slider.get()
        
        if hasattr(self, 'cluster_print') and self.cluster_print is not None:
            self.cluster_print.destroy()
            del self.cluster_print

        # Get the selected clustering method
        cluster_result = self.cluster_result
        self.cluster_print = ctk.CTkLabel(self.box_frame_sub, text=f"Cluster: {cluster_result}")
        self.cluster_print.pack(side="top", padx=5, pady=(5, 0))
        
        # Set up the pipeline for different clustering algorithms
        if cluster_result == "K-mean":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", KMeans(n_clusters=k, random_state=0, n_init='auto'))
            ])
            self.k_slider.configure(state="normal")
            
        elif cluster_result == "DBSCAN":
            min_samples_value = 10
            eps_value = self.estimate_eps(min_samples_value)
            
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", DBSCAN(eps=eps_value, min_samples=min_samples_value))
            ])
            self.k_slider.configure(state="disabled")
            
        elif cluster_result == "Mean Shift":  
            from sklearn.cluster import estimate_bandwidth
            bandwidth_value = estimate_bandwidth(self.loadings, quantile=0.2)
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", MeanShift(bandwidth=bandwidth_value))
            ])
            self.k_slider.configure(state="disabled")
            
        elif cluster_result == "Spectral":  
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=0))
            ])
            self.k_slider.configure(state="normal")
            
        elif cluster_result == "GMM":   
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", GaussianMixture(n_components=k, random_state=0))
            ])
            self.k_slider.configure(state="normal")
            
        elif cluster_result == "Affinity Propagation":   
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", AffinityPropagation(random_state=0))
            ])
            self.k_slider.configure(state="disabled")
            
        elif cluster_result == "Hierarchical":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", AgglomerativeClustering(n_clusters=k, linkage='ward'))
            ])
            self.k_slider.configure(state="normal")
            
        elif cluster_result == "BIRCH":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", Birch(n_clusters=k))
            ])
            self.k_slider.configure(state="normal")

        # Fit the pipeline to the data
        self.pipe.fit(self.loadings)
        self.clustering_results = pd.DataFrame(index=self.loadings.index)
        
        # Get clustering results
        if cluster_result == "GMM":  
            self.clustering_results['cluster'] = self.pipe.named_steps["model"].predict(self.loadings)
        else:
            self.clustering_results['cluster'] = self.pipe.named_steps['model'].labels_

        fig, axes = plt.subplots(ncols=len(self.loadings.columns), figsize=(15, 8))
        
        cluster_colors = self.clustering_results['cluster'].values
        self.clustering_results['element'] = self.loadings.index
        
        graph_data = []
        sort_clusters = self.cluster_sort_var.get() == 1
        
        for i in range(len(self.loadings.columns)):
            pc_name = f'PC{i+1}'
            data = self.loadings[pc_name]

            if sort_clusters:
                # Sort data by clusters
                temp_df = pd.DataFrame({
                    'data': data,
                    'cluster': cluster_colors,
                    'element':  self.clustering_results['element'],
                    'PC': pc_name
                })

                sorted_df = temp_df.sort_values(by='cluster')
                sorted_data = sorted_df['data']
                sorted_cluster_colors = sorted_df['cluster'].values
                sorted_labels = sorted_df['element'].values
                self.plot_pca_barchart(sorted_data, pc_name, axes[i], sorted_cluster_colors, sorted_labels)
                graph_data.append(sorted_df)
                
            else:
                temp_df = pd.DataFrame({
                    'data': data,
                    'cluster': cluster_colors,
                    'element':  self.clustering_results['element'],
                    'PC': pc_name
                })
                graph_data.append(temp_df)
                labels = temp_df['element'].values
                self.plot_pca_barchart(data, pc_name, axes[i], cluster_colors, labels)

        self.graph_data_df = pd.concat(graph_data)

        plt.tight_layout()
        
        # Add the plot to the tkinter canvas
        canvas = FigureCanvasTkAgg(fig, master=content_frame)
        canvas.draw()

        # Add navigation toolbar to the canvas
        toolbar = NavigationToolbar2Tk(canvas, content_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)

        self.yellowbrick()

    def save_loadings_to_excel(self):
        # Save the loadings to an Excel file
        if self.graph_data_df is None:
            print("No data to save. Ensure clusters are calculated and graph is generated.")
            return
        
        # Prompt user to select save location and filename
        file_name = fd.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
        if not file_name:
            return

        # Create a pivot table with the clustering results
        pivot_data = self.graph_data_df.pivot_table(index='element', columns='PC', values='data', sort=False)
        pivot_data.columns = [f'{col}' for col in pivot_data.columns]

        # Save the pivot table to an Excel file
        with pd.ExcelWriter(file_name, engine='openpyxl') as excel_writer:
            pivot_data.to_excel(excel_writer, index=True)

        # Load the saved workbook and worksheet
        wb = load_workbook(file_name)
        ws = wb.active

        # Define color map for clusters
        colormap = plt.get_cmap('Set2')
        cluster_colors = ['FF{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255)) for r, g, b, _ in [colormap(i) for i in range(colormap.N)]]
        cluster_fill = [PatternFill(start_color=color, end_color=color, fill_type="solid") for color in cluster_colors]

        # Apply cluster colors to the Excel file
        for i in range(len(self.graph_data_df)):
            element = self.graph_data_df.iloc[i]['element']
            cluster = self.graph_data_df.iloc[i]['cluster']
            row = ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1)
            for cell in row:
                if cell[0].value == element:
                    for col in range(2, ws.max_column + 1):
                        ws.cell(row=cell[0].row, column=col).fill = cluster_fill[cluster]

        wb.save(file_name)
        print(f"Loadings saved to {file_name}")

    def yellowbrick(self):
        # Create and display the KElbowVisualizer for clustering
        plt.rcParams.update({'font.size': 10})
        
        fig = plt.figure(figsize=(3, 2))
        ax = fig.add_subplot(111)
        ax.axis('off')
        
        model = self.pipe.named_steps['model']
        if 'n_clusters' in model.get_params():
            del model.get_params()['n_clusters']
    
        # Create and fit the KElbowVisualizer
        visualizer = KElbowVisualizer(model, k=(1, 12), timing=False, title="KElbowVisualizer").fit(self.loadings)
        
        visualizer.ax.set_yticklabels([])
        visualizer.finalize()
        
        # Add the plot to the tkinter canvas
        self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
        toolbar = CustomToolbar(self.canvas1, self.legend_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        
        plt.close(fig)


        





class sample_class:
    def __init__(self, shared_container, pca_df_scaled, df, cleaned_df, box_frame, box_frame_sub, on_button_click, apply_button):
        # Initialize the sample class with necessary parameters
        self.apply_button = apply_button
        self.shared_container = shared_container
        self.on_button_click = on_button_click
        self.pca_df_scaled = pca_df_scaled
        self.df = df
        self.cleaned_df = cleaned_df
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.var = IntVar()  # Variable for checkbox state
        self.Graph_PCA()  # Initialize the PCA graph

    def Graph_PCA(self):
        # Create button for displaying PC bar graph by samples
        pil_image = Image.open("images program/sample-screen-svgrepo-com.png")
        # resized_image = pil_image.resize((32, 32), Image.LANCZOS)
        self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))

        # Debug prints for loaded images
        print(pil_image)
        print(self.icon_image)

        # Create and place image button
        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text="", width=20)
        self.image_button.grid(row=1, column=1, sticky="w", pady=0, padx=5)

        # Bind button click event to PCA graph function
        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, self.Graph_PCA_sub))

        ToolTip(self.image_button, msg="PC Bar Graph by Samples")

    def Graph_PCA_sub(self):
        # Create UI for PCA graph options and actions
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        # Create and pack checkbox for sorting loadings
        self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Sort Loadings", variable=self.var)
        self.checkbox.pack(side="top", anchor="w", padx=5, pady=3)

        # Create and pack apply button
        self.apply_Graph_PCA = ctk.CTkButton(self.box_frame_sub, text="apply", command=self.update_plots)
        self.apply_Graph_PCA.pack(side="top", padx=5, pady=5)

    def normalize(self, values, vmin=None, vmax=None):
        # Normalize values for color scaling
        vmin = vmin if vmin is not None else np.min(values)
        vmax = vmax if vmin is not None else np.max(values)
        norm_values = (values - vmin) / (vmax - vmin)
        return norm_values

    def plot_pca_barchart(self, data, entry_name, ax, sort):
        # Plot the PCA bar chart
        if sort:
            data = data.sort_values(ascending=False)
        norm_values = self.normalize(data, vmin=-max(abs(data)), vmax=max(abs(data)))
        bar_colors = plt.cm.coolwarm(norm_values)
        data.plot(kind='barh', color=bar_colors, ax=ax, fontsize="xx-small")
        ax.set_title(entry_name)
        ax.grid(True, linestyle='--', linewidth=0.5)
        ax.set_ylabel('')

    def update_plots(self):
        # Update plots with selected options
        sort = self.var.get() == 1  # Get the state of the sort checkbox
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("Sample BarGraph")

        # Close all existing plots and clear widgets
        plt.close('all')
        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()

        # Determine the number of samples and PCs
        num_samples = len(self.pca_df_scaled)
        num_pcs = len(self.pca_df_scaled.columns)
        
        # Create subplots for each PC
        fig, axes = plt.subplots(ncols=num_pcs, figsize=(12, 10))

        for i in range(num_pcs):
            pc_name = f'PC{i+1}'
            data = self.pca_df_scaled.iloc[:, i]
            if sort:
                sorted_indices = data.sort_values(ascending=False).index
                data = data.loc[sorted_indices]
                y_tick_labels = self.cleaned_df.loc[sorted_indices, 'sample id']
            else:
                y_tick_labels = self.cleaned_df['sample id']

            # Plot PCA bar chart for each PC
            self.plot_pca_barchart(data, pc_name, axes[i], sort)
            axes[i].set_yticks(range(num_samples))
            axes[i].set_yticklabels(y_tick_labels, fontsize="xx-small")
            axes[i].set_ylabel('')

        plt.tight_layout()

        # Create and display the color map
        min_val = self.pca_df_scaled.min().min()
        max_val = self.pca_df_scaled.max().max()
        norm = plt.Normalize(vmin=min_val, vmax=max_val)
        cmap = plt.cm.coolwarm

        # Add the plot to the tkinter canvas
        canvas = FigureCanvasTkAgg(fig, master=content_frame)
        canvas.draw()

        # Add navigation toolbar to the canvas
        toolbar = NavigationToolbar2Tk(canvas, content_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)







class sample_cluster:
    def __init__(self, shared_container, cluster, pca_df_scaled, df, cleaned_df, box_frame, box_frame_sub, dc, ds, on_button_click, apply_button, legend_frame):
        # Initialize the PCCluster class with necessary parameters
        self.pca_df_scaled = pca_df_scaled
        self.shared_container = shared_container
        self.cluster_result = cluster
        self.df = df.dropna()
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.legend_frame = legend_frame
        self.cleaned_df = cleaned_df
        self.on_button_click = on_button_click
        self.apply_button = apply_button
        self.plot_cluster()
        self.dc = dc
        self.ds = ds
        self.var = IntVar()
        self.cluster_sort_var = IntVar()
        self.graph_data_df = None

    def plot_cluster(self):
        # Create button for displaying cluster bar graph by samples
        pil_image = Image.open("images program/id.png")
        # resized_image = pil_image.resize((32, 32), Image.LANCZOS)
        self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))

        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text = "", width=20)
        self.image_button.grid(row=3, column=1, sticky="w", pady=0, padx=5)

        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, self.plot_cluster_sub))

        ToolTip(self.image_button, msg="PC Cluster Bar Graph by Samples")
        
        # self.plot_cluster_button = ctk.CTkButton(self.box_frame, text="BG", command=lambda: self.on_button_click(self.plot_cluster_button, self.plot_cluster_sub), width=20)
        # self.plot_cluster_button.grid(row=3, column=3, sticky="w", pady=(5, 0), padx=0)

    def plot_cluster_sub(self):
        # Create UI for cluster options and actions
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()
        
        self.cluster_sort_checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Sort Clusters Together", variable=self.cluster_sort_var)
        self.cluster_sort_checkbox.pack(side="top", anchor="w", padx=5, pady=(3, 0))

        ctk.CTkLabel(self.box_frame_sub, text="Number of Clusters:").pack(side="top", padx=5, pady=(5, 0))
        self.k_slider = tk.Scale(self.box_frame_sub, from_=2, to=10, orient=tk.HORIZONTAL)
        self.k_slider.pack(side="top", padx=5, pady=(5, 0))

        self.apply_plot_3d_cluster = ctk.CTkButton(self.box_frame_sub, text="Apply", command=self.show_cluster)
        self.apply_plot_3d_cluster.pack(side="top", padx=5, pady=5)

        self.save_button = ctk.CTkButton(self.box_frame_sub, text="Save Clusters to Excel", command=self.save_clusters_to_excel)
        self.save_button.pack(side="bottom", padx=5, pady=5)
        

    def estimate_eps(self, k):
        # Estimate the optimal eps value for DBSCAN clustering
        from sklearn.neighbors import NearestNeighbors
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

        # Compute the k-nearest neighbors
        nearest_neighbors = NearestNeighbors(n_neighbors=k)
        nearest_neighbors.fit(self.X)
        distances, indices = nearest_neighbors.kneighbors(self.X)

        # Sort the distances (distances are ordered for each point)
        distances = np.sort(distances, axis=0)
        distances = distances[:, 1]  # Taking the distance to the k-th nearest neighbor

        # Plot the k-distance graph
        fig, ax = plt.subplots(figsize=(3, 2))
        ax.plot(distances)
        ax.set_title('k-Distance Graph')
        ax.set_xlabel('Points sorted by distance')
        ax.set_ylabel(f'{k}th Nearest Neighbor Distance')

        self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
        self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        plt.show()

        elbow_point_index = 80  
        eps_value = distances[elbow_point_index]
        return eps_value

    def show_cluster(self):
        # Display cluster analysis
        self.X = None
    
        # Create a new tab for displaying the PCA cluster by samples
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("PCA Cluster by Samples")
    
        # Close all existing plots and clear widgets
        plt.close('all')
        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
    
        # Get the number of clusters from the slider
        k = self.k_slider.get()
    
        # Get the scaled PCA values
        self.X = self.pca_df_scaled.values
    
        # Clear any existing cluster print label
        if hasattr(self, 'cluster_print') and self.cluster_print is not None:
            self.cluster_print.destroy()
            del self.cluster_print
    
        # Get the selected cluster method
        cluster_result = self.cluster_result
        self.cluster_print = ctk.CTkLabel(self.box_frame_sub, text=f"Cluster: {cluster_result}")
        self.cluster_print.pack(side="top", padx=5, pady=(5, 0))
    
        # Set up the pipeline for different clustering algorithms
        if cluster_result == "K-mean":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", KMeans(n_clusters=k, random_state=0, n_init='auto'))
            ])
            self.k_slider.configure(state="normal")
        elif cluster_result == "DBSCAN":
            min_samples_value = 10
            eps_value = self.estimate_eps(min_samples_value)
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", DBSCAN(eps=eps_value, min_samples=min_samples_value))
            ])
            self.k_slider.configure(state="disabled")
        elif cluster_result == "Mean Shift":
            from sklearn.cluster import estimate_bandwidth
            bandwidth_value = estimate_bandwidth(self.X, quantile=0.2)
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", MeanShift(bandwidth=bandwidth_value))
            ])
            self.k_slider.configure(state="disabled")
        elif cluster_result == "Spectral":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=0))
            ])
            self.k_slider.configure(state="normal")
        elif cluster_result == "GMM":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", GaussianMixture(n_components=k, random_state=0))
            ])
            self.k_slider.configure(state="normal")
        elif cluster_result == "Affinity Propagation":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", AffinityPropagation(random_state=0))
            ])
            self.k_slider.configure(state="disabled")
        elif cluster_result == "Hierarchical":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", AgglomerativeClustering(n_clusters=k, linkage='ward'))
            ])
            self.k_slider.configure(state="normal")
        elif cluster_result == "BIRCH":
            self.pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", Birch(n_clusters=k))
            ])
            self.k_slider.configure(state="normal")
    
        # Fit the pipeline to the data
        self.pipe.fit(self.X)
    
        # Assign cluster labels to the dataframe
        if cluster_result == "GMM":
            self.df['cluster'] = self.pipe.named_steps["model"].predict(self.X)
        else:
            self.df['cluster'] = self.pipe.named_steps['model'].labels_
    
        # Create the KElbowVisualizer
        self.yellowbrick()
    
        # Plot PCA components with clusters
        fig, axes = plt.subplots(nrows=1, ncols=self.X.shape[1], figsize=(15, 8))
        cluster_colors = self.df['cluster'].values
        sort_clusters = self.cluster_sort_var.get() == 1
        graph_data = []
    
        for i in range(self.X.shape[1]):
            pc_name = f'PC{i+1}'
            data = self.pca_df_scaled.iloc[:, i]
    
            if sort_clusters:
                temp_df = pd.DataFrame({
                    'data': data,
                    'cluster': cluster_colors,
                    'sample id': self.cleaned_df['sample id']
                })
                sorted_df = temp_df.sort_values(by='cluster')
                sorted_data = sorted_df['data']
                sorted_cluster_colors = sorted_df['cluster'].values
                sorted_labels = sorted_df['sample id'].values
                self.plot_pca_barchart(sorted_data, pc_name, axes[i], sorted_cluster_colors, sorted_labels)
                sorted_df['PC'] = pc_name
                graph_data.append(sorted_df)
            else:
                temp_df = pd.DataFrame({
                    'data': data,
                    'cluster': cluster_colors,
                    'sample id': self.cleaned_df['sample id'],
                    'PC': pc_name
                })
                graph_data.append(temp_df)
                labels = self.cleaned_df['sample id'].values
                self.plot_pca_barchart(data, pc_name, axes[i], cluster_colors, labels)
    
        # Combine graph data for saving to Excel
        self.graph_data_df = pd.concat(graph_data)
    
        # Adjust layout and display the plot
        plt.tight_layout()
        canvas = FigureCanvasTkAgg(fig, master=content_frame)
        canvas.draw()
        toolbar = NavigationToolbar2Tk(canvas, content_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)
    
    def plot_pca_barchart(self, data, entry_name, ax, cluster_colors, sorted_labels):
        # Plot the PCA bar chart with clusters
        cluster_palette = plt.get_cmap('tab20', len(np.unique(cluster_colors)))
        colors = cluster_palette(cluster_colors)
        data.plot(kind='barh', color=colors, ax=ax, fontsize="xx-small")
        ax.set_title(entry_name)
        ax.grid(True, linestyle='--', linewidth=0.5)
        ax.set_yticks(range(len(data)))
        ax.set_yticklabels(sorted_labels, fontsize="xx-small")
        ax.set_ylabel('')
    
    def yellowbrick(self):
        # Create and display the KElbowVisualizer for clustering
        plt.rcParams.update({'font.size': 10})
        fig = plt.figure(figsize=(3, 2))
        ax = fig.add_subplot(111)
        ax.axis('off')
        try:
            model = self.pipe.named_steps['model']
            if 'n_clusters' in model.get_params():
                del model.get_params()['n_clusters']
    
            X = self.pca_df_scaled.values
            visualizer = KElbowVisualizer(model, k=(1, 12), timing=False, title="KElbowVisualizer").fit(X)
    
            visualizer.ax.set_yticklabels([])
            visualizer.finalize()
    
            self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
            toolbar = CustomToolbar(self.canvas1, self.legend_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)
            self.canvas1.get_tk_widget().pack(fill="both", expand=True)
    
            plt.close(fig)
        except:
            pass
    
    def save_clusters_to_excel(self):
        # Save the clusters to an Excel file
        if self.graph_data_df is None:
            print("No data to save. Ensure clusters are calculated and graph is generated.")
            return
    
        # Prompt user to save the file
        file_name = fd.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
        if not file_name:
            return
    
        # Pivot the data for saving
        pivot_data = self.graph_data_df.pivot_table(index='sample id', columns='PC', values='data', sort=False)
        pivot_data.columns = [f'{col}' for col in pivot_data.columns]
    
        # Save the pivoted data to an Excel file
        with pd.ExcelWriter(file_name, engine='openpyxl') as excel_writer:
            pivot_data.to_excel(excel_writer, index=True)
    
        # Load the workbook and worksheet
        wb = load_workbook(file_name)
        ws = wb.active
    
        # Apply cluster colors to the cells
        colormap = plt.get_cmap('Set2')
        cluster_colors = ['FF{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255)) for r, g, b, _ in [colormap(i) for i in range(colormap.N)]]
        cluster_fill = [PatternFill(start_color=color, end_color=color, fill_type="solid") for color in cluster_colors]
    
        for i in range(len(self.graph_data_df)):
            sample_id = self.graph_data_df.iloc[i]['sample id']
            cluster = self.graph_data_df.iloc[i]['cluster']
            row = ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1)
            for cell in row:
                if cell[0].value == sample_id:
                    for col in range(2, ws.max_column + 1):
                        ws.cell(row=cell[0].row, column=col).fill = cluster_fill[cluster]
    
        # Save the workbook
        wb.save(file_name)
        print(f"Clusters saved to {file_name}")
    

                





class threed_class:
    def __init__(self, shared_container, pca_df_scaled, df, cleaned_df, box_frame, box_frame_sub, dc, ds, color_map, color_map1, on_button_click, apply_button, legend_frame, loadings):
        # Initialize the 3D class with necessary parameters
        self.apply_button = apply_button
        self.shared_container = shared_container
        self.on_button_click = on_button_click
        self.pca_df_scaled = pca_df_scaled
        self.df = df
        self.cleaned_df = cleaned_df
        self.dc = dc
        self.ds = ds
        self.color_map = color_map
        self.color_map1 = color_map1
        self.loadings = loadings
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.legend_frame = legend_frame
        self.var = IntVar()
        self.var1 = IntVar()
        self.var2 = IntVar()
        self.plot_3d()

    def plot_3d(self):
        # Create button for displaying 3D PCA biplot
        pil_image = Image.open("images program/cube-3d-svgrepo-com.png")
        # resized_image = pil_image.resize((32, 32), Image.LANCZOS)
        self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))
        
        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text="", width=20)
        self.image_button.grid(row=1, column=2, sticky="w", pady=0, padx=5)

        # Bind button click to plot_3d_sub function
        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, self.plot_3d_sub))

        ToolTip(self.image_button, msg="PCA 3D Biplot")

    def plot_3d_sub(self):
        # Create UI for selecting PCA components and options for 3D plot
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        pc_options = [f'PC{i+1}' for i in range(len(self.pca_df_scaled.T))]

        # Dropdowns for selecting PCA components
        ttk.Label(self.box_frame_sub, text="Select X-Axis:").pack(side="top", padx=5, pady=(5, 0))
        self.pc1_combo = ttk.Combobox(self.box_frame_sub, values=pc_options)
        self.pc1_combo.current(0)
        self.pc1_combo.pack(side="top", padx=5, pady=(5, 0))

        ttk.Label(self.box_frame_sub, text="Select Y-Axis:").pack(side="top", padx=5, pady=(5, 0))
        self.pc2_combo = ttk.Combobox(self.box_frame_sub, values=pc_options)
        self.pc2_combo.current(1)
        self.pc2_combo.pack(side="top", padx=5, pady=(5, 0))

        ttk.Label(self.box_frame_sub, text="Select Z-Axis:").pack(side="top", padx=5, pady=(5, 0))
        self.pc3_combo = ttk.Combobox(self.box_frame_sub, values=pc_options)
        self.pc3_combo.current(2)
        self.pc3_combo.pack(side="top", padx=5, pady=(5, 0))
        
        # Checkbox for displaying shapes
        if not self.ds.empty:
            self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display shape", variable=self.var)
            self.checkbox.pack(side="top", anchor="w", padx=5, pady=(3, 0))

        # Checkbox for displaying trendlines
        self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display Trendlines", variable=self.var2)
        self.checkbox.pack(side="top", anchor="w", padx=5, pady=(3, 0))

        # Checkbox for displaying legend
        self.checkbox1 = ctk.CTkCheckBox(self.box_frame_sub, text="Display Legend", variable=self.var1)
        self.checkbox1.pack(side="top", anchor="w", padx=5, pady=(3, 0))

        # Listbox for multiselect options
        self.multiselect_3d = tk.Listbox(self.box_frame_sub, selectmode=tk.MULTIPLE)
        self.multiselect_3d.pack(side="top", padx=5, pady=5)
        for names in self.df.columns.tolist():
            self.multiselect_3d.insert(tk.END, names)

        # Deselect all items in the listbox
        def deselect_all():
            self.multiselect_3d.selection_clear(0, tk.END)

        self.multiselect_3d.bind('<Command-d>', lambda event: deselect_all())

        # Apply button to show the 3D plot
        self.apply_plot_3d = ctk.CTkButton(self.box_frame_sub, text="apply", command=self.show_shape)
        self.apply_plot_3d.pack(side="top", padx=5, pady=5)
        
        # Display message for clicking data point if 'sample id' column exists
        if "sample id" in self.cleaned_df.columns:
            ctk.CTkLabel(self.box_frame_sub, text="Click Data Point for\n Sample ID").pack(side="top", padx=5, pady=(5, 0))

    def show_shape(self):
        # Show 3D PCA plot with selected components and options         
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("3D Biplot")

        plt.close('all')
        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

        pc1 = self.pc1_combo.get()
        pc2 = self.pc2_combo.get()
        pc3 = self.pc3_combo.get()

        fig = plt.figure(figsize=(11, 11))
        self.ax = fig.add_subplot(111, projection='3d')
        xdata = self.pca_df_scaled[pc1]
        ydata = self.pca_df_scaled[pc2]
        zdata = self.pca_df_scaled[pc3]

        canvas = FigureCanvasTkAgg(fig, master=content_frame)
        canvas.draw()

        toolbar = NavigationToolbar2Tk(canvas, content_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)

        # Plot the data points in 3D space
        if not self.dc.empty and not self.ds.empty:
            self.shapes = []
            for i in range(len(self.dc)):
                shape = self.ax.scatter3D(xdata[i], ydata[i], zdata[i], c=self.dc['Color'][i], marker=self.ds["Shapes"][i])
                self.shapes.append(shape)
    
            self.dots = self.ax.scatter3D(xdata, ydata, zdata, c=self.dc['Color'])
        else:
            self.dots = self.ax.scatter3D(xdata, ydata, zdata)

        # Plot trendlines if selected
        def plot_trendlines_3d():
            trendline = self.var2.get() == 1
            if trendline and 'lithology' in self.cleaned_df.columns:
                from mpl_toolkits.mplot3d import Axes3D
                from sklearn.linear_model import LinearRegression
                lithology_groups = self.cleaned_df.groupby('lithology')
                for name, group in lithology_groups:
                    x = self.pca_df_scaled.loc[group.index, pc1]
                    y = self.pca_df_scaled.loc[group.index, pc2]
                    z = self.pca_df_scaled.loc[group.index, pc3]
                    
                    X = np.vstack((x, y)).T
                    Y = z
                    
                    model = LinearRegression().fit(X, Y)
                    
                    trend_x = np.linspace(x.min(), x.max(), 100)
                    trend_y = np.linspace(y.min(), y.max(), 100)
                    trend_z = model.predict(np.vstack((trend_x, trend_y)).T)
                    color = self.color_map.get(name)
                    
                    self.ax.plot(trend_x, trend_y, trend_z, linestyle='--', label=f"Trendline {name}", color=color)
        
        plot_trendlines_3d()

        # Display sample IDs on click if selected
        if "sample id" in self.cleaned_df.columns:
            description = self.cleaned_df["sample id"]
            self.annotations = {}
            self.selected_points = set()
            
            from mpl_toolkits.mplot3d import proj3d
            
            def add_annotation(ind, source):
                if source == self.dots:
                    index = ind["ind"][0]
                    pos = source.get_offsets()[index]
                    text = f"{description.iloc[index]}"
                else:
                    index = ind
                    x, y, _ = proj3d.proj_transform(self.pca_df_scaled.iloc[index][pc1], self.pca_df_scaled.iloc[index][pc2], self.pca_df_scaled.iloc[index][pc3], self.ax.get_proj())
                    pos = (x, y)
                    text = f"{description.iloc[index]}"

                offsets = [(20, 20), (-20, 20), (20, -20), (-20, -20), (40, 40), (-40, 40), (40, -40), (-40, -40)]
                
                for offset in offsets:
                    annot = self.ax.annotate(text, xy=pos, xytext=offset,
                                        textcoords="offset points",
                                        bbox=dict(boxstyle="round", fc="w"),
                                        arrowprops=dict(arrowstyle="->",),
                                        zorder=100)
                    fig.canvas.draw()
                    bbox = annot.get_window_extent()
                    overlap = False
                    for existing_annot in self.annotations.values():
                        existing_bbox = existing_annot.get_window_extent()
                        if bbox.overlaps(existing_bbox):
                            overlap = True
                            annot.remove()
                            break
                    if not overlap:
                        annot.set_visible(True)
                        self.annotations[index] = annot
                        self.selected_points.add(index)
                        fig.canvas.draw_idle()
                        return 
            
                annot = self.ax.annotate(text, xy=pos, xytext=offsets[0],
                                    textcoords="offset points",
                                    bbox=dict(boxstyle="round", fc="w"),
                                    arrowprops=dict(arrowstyle="->"),
                                    zorder=100)
                annot.set_visible(True)
                self.annotations[index] = annot
                self.selected_points.add(index)
                fig.canvas.draw_idle()
            
            def remove_annotation(index):
                annot = self.annotations.pop(index, None)
                if annot:
                    annot.remove()
                self.selected_points.discard(index)
                fig.canvas.draw_idle()
            
            def update_annotations(event):
                for index, annot in self.annotations.items():
                    if index in self.selected_points:
                        x, y, _ = proj3d.proj_transform(self.pca_df_scaled.iloc[index][pc1], self.pca_df_scaled.iloc[index][pc2], self.pca_df_scaled.iloc[index][pc3], self.ax.get_proj())
                        annot.xy = (x, y)
                fig.canvas.draw_idle()
            
            def on_click(event):
                if event.inaxes == self.ax and event.button == MouseButton.LEFT:
                    contains_dots, ind_dots = self.dots.contains(event)
                    contains_shapes = False
                    shape_index = None
                    for i, shape in enumerate(self.shapes):
                        contains_shapes, ind_shapes = shape.contains(event)
                        if contains_shapes:
                            shape_index = i
                            break
                    
                    if contains_dots:
                        index = ind_dots["ind"][0]
                        if index in self.selected_points:
                            remove_annotation(index)
                        else:
                            add_annotation(ind_dots, self.dots)
                    elif contains_shapes:
                        if shape_index in self.selected_points:
                            remove_annotation(shape_index)
                        else:
                            add_annotation(shape_index, shape)
            
            fig.canvas.mpl_connect("button_press_event", on_click)
            fig.canvas.mpl_connect("motion_notify_event", update_annotations)

        self.setups(pc1, pc2, pc3)
        self.shape()
        self.legend()

    def shape(self):
        # Toggle visibility of shapes in the plot
        try:
            shape = self.var.get() == 1
            if shape:
                for shape in self.shapes:
                    shape.set_visible(True)
                self.dots.set_visible(False)
            else:
                self.dots.set_visible(True)
                for shape in self.shapes:
                    shape.set_visible(False)
        except:
            self.dots.set_visible(True)
            
    def legend(self):
        # Display legend for the plot
        if not self.dc.empty and not self.ds.empty:
            
            unique_lithologies = self.cleaned_df['lithology'].unique()
            unique_shapes = self.cleaned_df['rock unit'].unique()
    
            handle1 = []
            label1 = []
            for lithology in unique_lithologies:
                color = self.color_map[lithology]
                l1 = self.ax.scatter([], [], c=color, label=lithology)
                handle1.append(l1)
                label1.append(lithology)
            
            handle2 = []
            label2 = []
            for shape in unique_shapes:
                marker = self.color_map1[shape]
                l2 = self.ax.scatter([], [], c="black", marker=marker, label=shape)
                handle2.append(l2)
                label2.append(shape)
    
            total_entries = len(label1) + len(label2)
            height_per_entry = 0.225
            fig_height = total_entries * height_per_entry
    
            figx = plt.figure(figsize=(2.5, fig_height))
            axx = figx.add_subplot(111)
            axx.axis('off')
    
            self.legend1 = plt.legend(handle1, label1, bbox_to_anchor=(0.9, 1.15), title='Lithology', fontsize=10, labelspacing=0.3)
            self.legend2 = plt.legend(handle2, label2, bbox_to_anchor=(0.9, 0.35), title='Rock Type', fontsize=10, labelspacing=0.3)

            axx.add_artist(self.legend1)
            axx.add_artist(self.legend2)
            self.canvas1 = FigureCanvasTkAgg(figx, master=self.legend_frame)
            
            toolbar = CustomToolbar(self.canvas1, self.legend_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)

            self.canvas1.get_tk_widget().pack(fill="both", expand=True)
            
            self.legend1.set_visible(False)
            self.legend2.set_visible(False)
    
            Legend = self.var1.get() == 1
            if Legend:
                self.legend1.set_visible(True)
                self.legend2.set_visible(True)
            else:
                self.legend1.set_visible(False)
                self.legend2.set_visible(False)
        else:
            pass

    def setups(self, pc1, pc2, pc3):
        # Set up 3D biplot with loadings
        show_names = self.loadings.index.tolist()
        indx = self.loadings.index.get_indexer(show_names)
        scale = 1

        xs = scale * self.loadings[pc1]
        ys = scale * self.loadings[pc2]
        zs = scale * self.loadings[pc3]

        plt.title(f'3D Biplot')

        self.ax.set_xlabel(pc1)
        self.ax.set_ylabel(pc2)
        self.ax.set_zlabel(pc3)
        x_arr = np.zeros(len(self.loadings[pc1]))
        y_arr = z_arr = x_arr

        all_points = np.concatenate((xs, ys, zs))
        min_value = np.min(all_points)
        max_value = np.max(all_points)
        axis_range = [min_value, max_value]

        self.ax.plot([x_arr.any() + axis_range[0], x_arr.any() + axis_range[1]], [y_arr.any(), y_arr.any()], [z_arr.any(), z_arr.any()], color='k', linestyle='--')
        self.ax.plot([x_arr.any(), x_arr.any()], [y_arr.any() + axis_range[0], y_arr.any() + axis_range[1]], [z_arr.any(), z_arr.any()], color='k', linestyle='--')
        self.ax.plot([x_arr.any(), x_arr.any()], [y_arr.any(), y_arr.any()], [z_arr.any() + axis_range[0], z_arr.any() + axis_range[1]], color='k', linestyle='--')

        self.arrow_list = []
        for i, name in enumerate(self.df.columns):
            ip = self.df.columns.get_loc(name)
            arrow = self.ax.quiver(x_arr[ip], y_arr[ip], z_arr[ip], xs[ip], ys[ip], zs[ip], color='r', arrow_length_ratio=0.1, linewidth=0.5)
            self.arrow_list.append(arrow)

        self.namelist = []
        for i, names in enumerate(show_names):
            ip = indx[i]
            n = self.ax.text(xs[ip] + 0.02, ys[ip] + 0.02, zs[ip] + 0.02, names, fontsize='small')
            self.namelist.append(n)

        selected_indices = self.multiselect_3d.curselection()
        self.selected_items = [self.multiselect_3d.get(i) for i in selected_indices]
        self.findname_specific()

    def findname_specific(self):
        # Highlight selected names and arrows in the plot
        options = self.df.columns.tolist()
        for i in range(len(options[:])):
            if options[i] in self.selected_items:
                self.namelist[i].set_visible(True)
                self.arrow_list[i].set_visible(True)
            else:
                self.namelist[i].set_visible(False)
                self.arrow_list[i].set_visible(False)






    


class twod_class:
    def __init__(self, shared_container, pca_df_scaled, df, cleaned_df, box_frame, box_frame_sub, dc, ds, color_map, color_map1, on_button_click, apply_button, legend_frame, loadings):
        # Initialize the 2D class with required parameters
        self.shared_container = shared_container
        self.pca_df_scaled = pca_df_scaled
        self.df = df
        self.cleaned_df = cleaned_df
        self.dc = dc
        self.ds = ds
        self.color_map = color_map
        self.color_map1 = color_map1
        self.loadings = loadings

        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.legend_frame = legend_frame

        self.var = IntVar()
        self.var1 = IntVar()
        self.var2 = IntVar()
        self.on_button_click = on_button_click
        self.apply_button = apply_button
        
        self.plot_2d()

    def plot_2d(self):
        # Load and display the 2D plot icon button
        pil_image = Image.open("images program/chart-scatterplot-svgrepo-com.png")
        resized_image = pil_image.resize((32, 32), Image.LANCZOS)
        self.icon_image = ImageTk.PhotoImage(resized_image)
        
        print(pil_image)
        print(self.icon_image)
        
        # Create a label with the image button
        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text="", width=20)
        self.image_button.grid(row=1, column=3, sticky="w", pady=0, padx=5)
        
        # Bind the button click to the sub plot function
        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, self.plot_2d_sub))

        ToolTip(self.image_button, msg="PCA 2D Biplot")

    def plot_2d_sub(self):
        # Clear any existing widgets in the sub-frame
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        # Options for selecting principal components
        pc_options = [f'PC{i+1}' for i in range(len(self.pca_df_scaled.T))]
        
        # Dropdown for selecting the first principal component
        ctk.CTkLabel(self.box_frame_sub, text="Select PC1:").pack(side="top", padx=5, pady=(5, 0))
        self.pc1_combo = ttk.Combobox(self.box_frame_sub, values=pc_options)
        self.pc1_combo.current(0)
        self.pc1_combo.pack(side="top", padx=5, pady=(5, 0))
        
        # Dropdown for selecting the second principal component
        ctk.CTkLabel(self.box_frame_sub, text="Select PC2:").pack(side="top", padx=5, pady=(5, 0))
        self.pc2_combo = ttk.Combobox(self.box_frame_sub, values=pc_options)
        self.pc2_combo.current(1)
        self.pc2_combo.pack(side="top", padx=5, pady=(5, 0))
        
        # Checkbox for displaying shapes
        if not self.ds.empty:
            self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display shape", variable=self.var)
            self.checkbox.pack(side="top", anchor="w", padx=5, pady=(3, 0))

        # Checkbox for displaying trendlines
        self.checkbox2 = ctk.CTkCheckBox(self.box_frame_sub, text="Display Trendlines", variable=self.var2)
        self.checkbox2.pack(side="top", anchor="w", padx=5, pady=(3, 0))
        
        # Checkbox for displaying legend
        self.checkbox1 = ctk.CTkCheckBox(self.box_frame_sub, text="Display Legend", variable=self.var1)
        self.checkbox1.pack(side="top", anchor="w", padx=5, pady=(3, 0))

        # Multiselect listbox for selecting columns
        self.multiselect_2d = tk.Listbox(self.box_frame_sub, selectmode=tk.MULTIPLE)
        self.multiselect_2d.pack(side="top", padx=5, pady=5)
        for names in self.df.columns.tolist():
            self.multiselect_2d.insert(tk.END, names)
            
        # Function to deselect all items in the listbox
        def deselect_all():
            self.multiselect_2d.selection_clear(0, tk.END)

        # Bind Command-d (deselect all) to the listbox
        self.multiselect_2d.bind('<Command-d>', lambda event: deselect_all())

        # Apply button to show the 2D plot
        self.apply_plot_2d = ctk.CTkButton(self.box_frame_sub, text="apply", command=self.show_shape)
        self.apply_plot_2d.pack(side="top", padx=5, pady=5)
        
        # Display message for clicking data point if 'sample id' column exists
        if "sample id" in self.cleaned_df.columns:
            ctk.CTkLabel(self.box_frame_sub, text="Click Data Point for \n Sample ID").pack(side="top", padx=5, pady=(5, 0))

    def show_shape(self):            
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("2D Biplot")

        plt.close('all')
        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
    
        pc1 = self.pc1_combo.get()
        pc2 = self.pc2_combo.get()
    
        xdata = self.pca_df_scaled[pc1]
        ydata = self.pca_df_scaled[pc2]
        
        fig = plt.figure(figsize=(10, 10))
        
        # Create a subplot grid if 'lithology' column exists
        if 'lithology' in self.cleaned_df.columns:
            gs = fig.add_gridspec(5, 5, hspace=0.4, wspace=0.4)
            self.ax = fig.add_subplot(gs[1:5, 0:4])
            ax_box_x = fig.add_subplot(gs[0, 0:4], sharex=self.ax)
            ax_box_y = fig.add_subplot(gs[1:5, 4], sharey=self.ax)
        else:
            self.ax = fig.add_subplot(111)
        
        # Plot data points with shapes and colors if available
        if not self.dc.empty and not self.ds.empty:
            self.shapes = []
            for i in range(len(self.df)):
                shape = self.ax.scatter(xdata[i], ydata[i], c=self.dc['Color'][i], marker=self.ds["Shapes"][i])
                self.shapes.append(shape)
                
            self.dots = self.ax.scatter(xdata, ydata, c=self.dc['Color'])
        else:
            self.dots = self.ax.scatter(xdata, ydata)

        def plot_trendlines():
            # Plot trendlines for lithology groups
            trendline = self.var2.get() == 1
            if trendline and 'lithology' in self.cleaned_df.columns:
                import statsmodels.api as sm
                lithology_groups = self.cleaned_df.groupby('lithology')
                for name, group in lithology_groups:
                    x = self.pca_df_scaled.loc[group.index, pc1]
                    y = self.pca_df_scaled.loc[group.index, pc2]
                    
                    color = self.color_map.get(name)
                    
                    sns.regplot(x=x, y=y, ax=self.ax, scatter=False, label=f"Trendline {name}", color=color, ci=50, line_kws={'linewidth': 1})
        
        # Call the plot_trendlines function if enabled
        plot_trendlines()

        if "sample id" in self.cleaned_df.columns:
            # Annotate sample IDs on click
            description = self.cleaned_df["sample id"]
            self.annotations = {}
            self.selected_points = set()
            
            def add_annotation(ind, source):
                if source == self.dots:
                    index = ind["ind"][0]
                    pos = source.get_offsets()[index]
                    text = f"{description.iloc[index]}"
                else:
                    index = ind
                    pos = (self.pca_df_scaled.iloc[index][pc1], self.pca_df_scaled.iloc[index][pc2])
                    text = f"{description.iloc[index]}"
            
                offsets = [(20, 20), (-20, 20), (20, -20), (-20, -20), (40, 40), (-40, 40), (40, -40), (-40, -40)]
                
                # Try different annotation positions to avoid overlap
                for offset in offsets:
                    annot = self.ax.annotate(text, xy=pos, xytext=offset,
                                        textcoords="offset points",
                                        bbox=dict(boxstyle="round", fc="w"),
                                        arrowprops=dict(arrowstyle="->"),
                                        zorder=100)
                    fig.canvas.draw()
                    bbox = annot.get_window_extent()
                    overlap = False
                    for existing_annot in self.annotations.values():
                        existing_bbox = existing_annot.get_window_extent()
                        if bbox.overlaps(existing_bbox):
                            overlap = True
                            annot.remove()
                            break
                    if not overlap:
                        annot.set_visible(True)
                        self.annotations[index] = annot
                        self.selected_points.add(index)
                        fig.canvas.draw_idle()
                        return 
            
                # If all positions overlap, place annotation in the default position
                annot = self.ax.annotate(text, xy=pos, xytext=offsets[0],
                                    textcoords="offset points",
                                    bbox=dict(boxstyle="round", fc="w"),
                                    arrowprops=dict(arrowstyle="->"),
                                    zorder=100)
                annot.set_visible(True)
                self.annotations[index] = annot
                self.selected_points.add(index)
                fig.canvas.draw_idle()
            
            def remove_annotation(index):
                # Remove annotation from the plot
                annot = self.annotations.pop(index, None)
                if annot:
                    annot.remove()
                self.selected_points.discard(index)
                fig.canvas.draw_idle()
            
            def update_annotations(event):
                # Update annotation positions on mouse move
                for index, annot in self.annotations.items():
                    if index in self.selected_points:
                        annot.xy = (self.pca_df_scaled.iloc[index][pc1], self.pca_df_scaled.iloc[index][pc2])
                fig.canvas.draw_idle()
            
            def on_click(event):
                # Handle mouse click to add or remove annotations
                if event.inaxes == self.ax and event.button == MouseButton.LEFT:
                    contains_dots, ind_dots = self.dots.contains(event)
                    contains_shapes = False
                    shape_index = None
                    for i, shape in enumerate(self.shapes):
                        contains_shapes, ind_shapes = shape.contains(event)
                        if contains_shapes:
                            shape_index = i
                            break
                    
                    if contains_dots:
                        index = ind_dots["ind"][0]
                        if index in self.selected_points:
                            remove_annotation(index)
                        else:
                            add_annotation(ind_dots, self.dots)
                    elif contains_shapes:
                        if shape_index in self.selected_points:
                            remove_annotation(shape_index)
                        else:
                            add_annotation(shape_index, shape)
            
            # Connect event handlers for clicking and moving the mouse
            fig.canvas.mpl_connect("button_press_event", on_click)
            fig.canvas.mpl_connect("motion_notify_event", update_annotations)

        if 'lithology' in self.cleaned_df.columns:
            # Create and display box plots for lithology
            filtered_df = self.cleaned_df.copy()
            
            xdata_filtered = self.pca_df_scaled.loc[filtered_df.index, pc1]
            ydata_filtered = self.pca_df_scaled.loc[filtered_df.index, pc2]
    
            filtered_colors = filtered_df['lithology'].map(self.color_map)
            
            unique_lithologies = filtered_df['lithology'].nunique()
            palette = [self.color_map[lithology] for lithology in filtered_df['lithology'].unique()]
            
            sns.boxplot(x=xdata_filtered, y=filtered_df['lithology'], ax=ax_box_x, palette=palette, orient='h')
            sns.boxplot(y=ydata_filtered, x=filtered_df['lithology'], ax=ax_box_y, palette=palette, orient='v')
    
            # Customize tick parameters for the box plots
            ax_box_x.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
            ax_box_x.tick_params(axis='y', labelsize=8, left=False, right=True, labelleft=False, labelright=True)
    
            ax_box_y.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
            ax_box_y.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    
            ax_box_x.set_ylabel(None)
            ax_box_x.set_xlabel(None)
            
            ax_box_y.set_ylabel(None)
            ax_box_y.set_xlabel(None)

            self.ax.axvline(x=0, color='black', linewidth=0.5)
            self.ax.axhline(y=0, color='black', linewidth=0.5)

        # Create and display the plot canvas and toolbar
        canvas = FigureCanvasTkAgg(fig, master=content_frame)
        canvas.draw()

        toolbar = NavigationToolbar2Tk(canvas, content_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)
        
        # Setup additional plot features
        self.setups(pc1, pc2)
        self.shape()
        self.legend()

    def shape(self):
        # Show or hide shapes based on checkbox selection
        try:
            shape = self.var.get() == 1
            if shape:
                for shape in self.shapes:
                    shape.set_visible(True)
                self.dots.set_visible(False)
            else:
                self.dots.set_visible(True)
                for shape in self.shapes:
                    shape.set_visible(False)
        except:
            self.dots.set_visible(True)
            
    def legend(self):  
        # Create and display legends for lithology and rock type
        if not self.dc.empty and not self.ds.empty:
            
            unique_lithologies = self.cleaned_df['lithology'].unique()
            unique_shapes = self.cleaned_df['rock unit'].unique()
    
            handle1 = []
            label1 = []
            for lithology in unique_lithologies:
                color = self.color_map[lithology]
                l1 = self.ax.scatter([], [], c=color, label=lithology)
                handle1.append(l1)
                label1.append(lithology)
            
            handle2 = []
            label2 = []
            for shape in unique_shapes:
                marker = self.color_map1[shape]
                l2 = self.ax.scatter([], [], c="black", marker=marker, label=shape)
                handle2.append(l2)
                label2.append(shape)
            
            total_entries = len(label1) + len(label2)
            height_per_entry = 0.225
            fig_height = total_entries * height_per_entry
                
            figx = plt.figure(figsize=(2.5, fig_height))
            axx = figx.add_subplot(111)
            axx.axis('off')
        
            self.legend1 = plt.legend(handle1, label1, bbox_to_anchor=(0.9, 1.15), title='Lithology', fontsize=10, labelspacing=0.3)
            self.legend2 = plt.legend(handle2, label2, bbox_to_anchor=(0.9, 0.35), title='Rock Type', fontsize=10, labelspacing=0.3)
            axx.add_artist(self.legend1)
            axx.add_artist(self.legend2)
            self.canvas1 = FigureCanvasTkAgg(figx, master=self.legend_frame)
            toolbar = CustomToolbar(self.canvas1, self.legend_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)

            self.canvas1.get_tk_widget().pack(fill="both", expand=True)
    
            self.legend1.set_visible(False)
            self.legend2.set_visible(False)
            self.legenda()
        else:
            pass
            
    def legenda(self):
        # Show or hide legends based on checkbox selection
        Legend = self.var1.get() == 1
        if Legend:
            self.legend1.set_visible(True)
            self.legend2.set_visible(True)
        else:
            self.legend1.set_visible(False)
            self.legend2.set_visible(False)

    def setups(self, pc1, pc2):
        # Setup for 2D biplot with arrows and labels for loadings
        show_names = self.loadings.index.tolist()
        indx = self.loadings.index.get_indexer(show_names)
        
        pc1_index = int(pc1[2:]) - 1
        pc2_index = int(pc2[2:]) - 1
        xs = self.loadings[pc1]
        ys = self.loadings[pc2]

        arrow_list_2d = []
        
        x_arr = np.zeros(len(self.loadings[pc1]))
        y_arr = x_arr

        max_x = max(abs(xs))
        max_y = max(abs(ys))
        arrow_scale = max(max_x, max_y) / 0.5
        
        for i, name in enumerate(self.df.columns):
            ip = self.df.columns.get_loc(name)
            arrow_2d = self.ax.quiver(x_arr[ip], y_arr[ip], xs[ip], ys[ip], color='r', scale=arrow_scale, width=0.002)
            arrow_list_2d.append(arrow_2d)
        
        namelist_2d = []
        for i, names in enumerate(show_names):
            ip = indx[i]
            n = self.ax.text((1 / arrow_scale) * xs[ip], (1 / arrow_scale) * ys[ip], names, fontsize='small')
            namelist_2d.append(n)

        selected_indices = self.multiselect_2d.curselection()
        self.selected_items = [self.multiselect_2d.get(i) for i in selected_indices]
        self.findname_specific_2d(namelist_2d, arrow_list_2d)
        
        self.ax.set_xlabel(pc1)
        self.ax.set_ylabel(pc2)
        self.ax.set_title(f"{pc1} vs {pc2}")

    def findname_specific_2d(self, namelist_2d, arrow_list_2d):
        # Show or hide specific names and arrows based on user selection
        options = self.df.columns.tolist()
        for i in range(len(options[:])):
            if options[i] in self.selected_items:
                namelist_2d[i].set_visible(True)
                arrow_list_2d[i].set_visible(True)
            else:
                namelist_2d[i].set_visible(False)
                arrow_list_2d[i].set_visible(False)









class Cluster2DPlotClass:
    def __init__(self, shared_container, cluster, df, cleaned_df, box_frame, box_frame_sub, dc, ds, color_map, color_map1, on_button_click, legend_frame):
        # Initialize the 2D cluster plot class with required parameters
        self.df = df.dropna()
        self.shared_container = shared_container
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.dc = dc
        self.ds = ds
        self.color_map = color_map
        self.color_map1 = color_map1
        self.on_button_click = on_button_click
        self.legend_frame = legend_frame
        self.cleaned_df = cleaned_df
        self.cleaned_df.columns = self.cleaned_df.columns.str.replace('_ppm', '').str.replace('_pct', '').str.replace('_Howell', '')
        self.var = IntVar()
        self.var1 = IntVar()
        self.cluster_result = cluster
        # Bind the cluster change event to update the plot
        # self.kmean_instance.scaler_combo.bind("<<ClusterChanged>>", self.on_cluster_change)
    
        self.plot_2d_cluster()

    # def on_cluster_change(self):
    #     # Handle cluster change event and update the plot
    #     self.cluster_result = self.kmean_instance.get_cluster()
    #     self.plot_2d_cluster_sub()

    def plot_2d_cluster(self):
        # Load and display the 2D cluster plot icon button
        pil_image = Image.open("images program/2d.png")
        self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))

        print(pil_image)
        print(self.icon_image)
        
        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text = "", width=20)
        self.image_button.grid(row=3, column=3, sticky="w", pady=0, padx=5)
        
        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, lambda: self.plot_2d_cluster_sub(self.cluster_result)))

        ToolTip(self.image_button, msg="PCA 2D Biplot with Clusters")
        
        # self.plot_2d_cluster_button = ctk.CTkButton(self.box_frame, text="2D", command=lambda: self.on_button_click(self.plot_2d_cluster_button, self.plot_2d_cluster_sub), width=20)
        # self.plot_2d_cluster_button.grid(row=3, column=1, sticky="w", pady=(5,0), padx=0)

        
    def plot_2d_cluster_sub(self, cluster):
        self.cluster_result = cluster
        # Display options for selecting axes and cluster settings
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()
            
        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 1:").grid(row=0, column=0, pady=(5,0), padx=5)
        self.axis1_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis1_combo.current(0)
        self.axis1_combo.grid(row=1, column=0, pady=(3,0), padx=5)
        
        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 2:").grid(row=2, column=0, pady=(5,0), padx=5)
        self.axis2_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis2_combo.current(1)
        self.axis2_combo.grid(row=3, column=0, pady=(3,0), padx=5)

        ctk.CTkLabel(self.box_frame_sub, text="Select Element for Data Point Size").grid(row=4, column=0, pady=(5,0), padx=5)
        self.size_combo1 = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist(), name="size_combo1")
        self.size_combo1.set("N/A")
        self.size_combo1.grid(row=5, column=0, pady=(3,5), padx=5)

        if not self.ds.empty:
            self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display shape", variable=self.var)
            self.checkbox.grid(row=6, column=0, sticky=W, pady=(5,0), padx=5)

        self.legend_checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Show Legend", variable=self.var1)
        self.legend_checkbox.grid(row=7, column=0, sticky=W, pady=(5,0), padx=5)
            
        self.cluster_text = ctk.CTkLabel(self.box_frame_sub, text="Number of Clusters:")
        self.k_slider = tk.Scale(self.box_frame_sub, from_=2, to=10, orient=tk.HORIZONTAL)

        if self.cluster_result in ["DBSCAN", "Mean Shift", "Affinity Propagation"]:
            self.cluster_text.grid_forget()
            self.k_slider.grid_forget()
        else:
            self.cluster_text.grid(row=8, column=0, pady=(5,0), padx=5)
            self.k_slider.grid(row=9, column=0, pady=(3,0), padx=5)

        self.apply_plot_2d_cluster = ctk.CTkButton(self.box_frame_sub, text="Apply", command=self.show_cluster)
        self.apply_plot_2d_cluster.grid(row=10, column=0, pady=(5,0), padx=5)

        self.save_button = ctk.CTkButton(self.box_frame_sub, text="Save Clusters to Excel", command=self.save_clusters_to_excel)
        self.save_button.pack(side="bottom", padx=5, pady=5)


    def yellowbrick(self):
        # Visualize the elbow method for cluster selection
        try:
            plt.rcParams.update({'font.size': 10})
            
            fig = plt.figure(figsize=(3, 2))
            ax = fig.add_subplot(111)
            ax.axis('off')
            
            model = self.pipe.named_steps['model']
            if 'n_clusters' in model.get_params():
                del model.get_params()['n_clusters']
            visualizer = KElbowVisualizer(model, k=(1, 12), timing=False, title="KElbowVisualizer").fit(self.X)
            
            visualizer.ax.set_xlabel("")
            visualizer.ax.set_ylabel("")
            visualizer.ax.set_yticklabels([])
            visualizer.ax.set_xticklabels([])
            
            visualizer.finalize()
            
            self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
            toolbar = CustomToolbar(self.canvas1, self.legend_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)
            self.canvas1.get_tk_widget().pack(fill="both", expand=True)
            
            plt.close(fig)
        except:
            pass

    def estimate_eps(self, k):
        # Estimate epsilon for DBSCAN using k-distance graph
        from sklearn.neighbors import NearestNeighbors
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

        # Compute the k-nearest neighbors
        nearest_neighbors = NearestNeighbors(n_neighbors=k)
        nearest_neighbors.fit(self.X)
        distances, indices = nearest_neighbors.kneighbors(self.X)

        # Sort the distances (distances are ordered for each point)
        distances = np.sort(distances, axis=0)
        distances = distances[:, 1]  # Taking the distance to the k-th nearest neighbor

        # Plot the k-distance graph
        fig, ax = plt.subplots(figsize=(3, 2))
        ax.plot(distances)
        ax.set_title('k-Distance Graph')
        ax.set_xlabel('Points sorted by distance')
        ax.set_ylabel(f'{k}th Nearest Neighbor Distance')


        self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
        self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        plt.show()

        elbow_point_index = 80  
        eps_value = distances[elbow_point_index]
        return eps_value

    def show_cluster(self):         
        # Check if there is an active tab in the shared container
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("2D Cluster")

        # Clear the content frame and legend frame
        plt.close('all')
        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()
            
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
        
        axis1 = self.axis1_combo.get()
        axis2 = self.axis2_combo.get()
        k = self.k_slider.get()

        # Ensure the selected axes are different
        if axis1 != axis2:
            self.df[axis1] = self.df[axis1].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x).astype(float)
            self.df[axis2] = self.df[axis2].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x).astype(float)
            
            self.X = self.df[[axis1, axis2]].values
            
            if hasattr(self, 'cluster_print') and self.cluster_print is not None:
                self.cluster_print.destroy()
                del self.cluster_print

            self.cluster_print = ctk.CTkLabel(self.box_frame_sub, text=f"Cluster: {self.cluster_result}")
            self.cluster_print.grid(row=11, column=0, pady=(5,0), padx=5)

            # Set up the clustering pipeline based on selected method
            if self.cluster_result == "K-mean":
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", KMeans(n_clusters=k, random_state=0, n_init='auto'))
                ])
                self.cluster_text.grid(row=8, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=9, column=0, pady=(3,0), padx=5)
                
            elif self.cluster_result == "DBSCAN":
                min_samples_value = 10
                eps_value = self.estimate_eps(min_samples_value)
                
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", DBSCAN(eps=eps_value, min_samples=min_samples_value))
                ])
                self.cluster_text.grid_forget()
                self.k_slider.grid_forget()
                
            elif self.cluster_result == "Mean Shift":  
                from sklearn.cluster import estimate_bandwidth
                bandwidth_value = estimate_bandwidth(self.X, quantile=0.2)
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", MeanShift(bandwidth=bandwidth_value))
                ])
                self.cluster_text.grid_forget()
                self.k_slider.grid_forget()
                
            elif self.cluster_result == "Spectral":  
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=0))
                ])
                self.cluster_text.grid(row=8, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=9, column=0, pady=(3,0), padx=5)
                
            elif self.cluster_result == "GMM":   
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", GaussianMixture(n_components=k, random_state=0))
                ])
                self.cluster_text.grid(row=8, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=9, column=0, pady=(3,0), padx=5)
                
            elif self.cluster_result == "Affinity Propagation":   
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", AffinityPropagation(random_state=0))
                ])
                self.cluster_text.grid_forget()
                self.k_slider.grid_forget()
                
            elif self.cluster_result == "Hierarchical":
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", AgglomerativeClustering(n_clusters=k, linkage='ward'))
                ])
                self.cluster_text.grid(row=8, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=9, column=0, pady=(3,0), padx=5)
                
            elif self.cluster_result == "BIRCH":
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", Birch(n_clusters=k))
                ])
                self.cluster_text.grid(row=8, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=9, column=0, pady=(3,0), padx=5)
    
            self.pipe.fit(self.X)
            
            if self.cluster_result == "GMM":  
                self.df['cluster'] = self.pipe.named_steps["model"].predict(self.X)
            else:
                self.df['cluster'] = self.pipe.named_steps['model'].labels_
            
            # Create a meshgrid for the plot
            try:
                rangex = (self.X[:, 0].max()-self.X[:, 0].min())/10
                rangey = (self.X[:, 1].max()-self.X[:, 1].min())/10
                x_min, x_max = self.X[:, 0].min() - rangex, self.X[:, 0].max() + rangex
                y_min, y_max = self.X[:, 1].min() - rangey, self.X[:, 1].max() + rangey
                print (rangex)
                print (rangey)
                
                stepsize = min([rangex, rangey])/70
                print (stepsize)
                xx, yy = np.meshgrid(np.arange(x_min, x_max, stepsize),
                                     np.arange(y_min, y_max, stepsize))
                
                if self.cluster_result == "Spectral" or self.cluster_result == "DBSCAN" or self.cluster_result == "Hierarchical":
                    self.pipe.named_steps['scale'].fit(self.X)
                    X_scaled = self.pipe.named_steps['scale'].transform(self.X)
                    self.pipe.named_steps['model'].fit(X_scaled)
            
                    def predict_new_data(new_data, pipe, train_data, train_labels):
                        new_data_scaled = pipe.named_steps['scale'].transform(new_data)
                        distances = np.linalg.norm(new_data_scaled[:, np.newaxis] - train_data[np.newaxis, :], axis=2)
                        nearest_indices = np.argmin(distances, axis=1)
                        return train_labels[nearest_indices]
            
                    labels = self.pipe.named_steps['model'].labels_
    
                    new_data = np.c_[xx.ravel(), yy.ravel()]
                    new_data_labels = predict_new_data(new_data, self.pipe, X_scaled, labels)
            
                    Z = new_data_labels.reshape(xx.shape)
    
                else:
                    Z = self.pipe.named_steps['model'].predict(self.pipe.named_steps['scale'].transform(np.c_[xx.ravel(), yy.ravel()]))
                    Z = Z.reshape(xx.shape)
    
                fig, ax = plt.subplots(figsize=(9, 7))
                ax.contourf(xx, yy, Z, cmap=plt.cm.Pastel1, alpha=0.2)
                
                def map_size(x, new_min, new_max):
                    old_min = self.min_ele
                    old_max = self.max_ele
                    return ((np.log(x + 1) - np.log(old_min + 1)) / (np.log(old_max + 1) - np.log(old_min + 1))) * (new_max - new_min) + new_min
            except:
                fig, ax = plt.subplots(figsize=(9, 7))

            self.shapes = []
            colors = self.dc['Color']

             # Plot the points with shapes and colors
            element_size = self.size_combo1.get()
            if element_size == "N/A":
                if "lithology" in self.cleaned_df.columns and "rock unit" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i],
                            color=colors[i], marker=self.ds["Shapes"][i]
                        )
                        self.shapes.append(shape)

                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], c=colors)

                elif "lithology" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i],
                            color=colors[i]
                        )
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], c=colors)                        

                elif "rock unit" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i],
                            marker=self.ds["Shapes"][i]
                        )
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2])                        

                else:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df.iloc[i][axis1], self.df.iloc[i][axis2])
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2])
            else:
                # Try to get element_size column values for size mapping
                self.max_ele = self.df[element_size].max()
                self.min_ele = self.df[element_size].min()
                
                # Function to map element_size values to point sizes
                def map_size(x, new_min, new_max):
                    old_min = self.min_ele
                    old_max = self.max_ele
                    return ((np.log(x + 1) - np.log(old_min + 1)) / (np.log(old_max + 1) - np.log(old_min + 1))) * (new_max - new_min) + new_min
                    
                sizes = self.df[element_size].apply(lambda x: map_size(x, 20, 100))

                if "lithology" in self.cleaned_df.columns and "rock unit" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i],
                            color=colors[i], marker=self.ds["Shapes"][i], s=sizes[i]
                        )
                        self.shapes.append(shape)   
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], c=colors, s=self.df[element_size].apply(lambda x: map_size(x, 20, 100)))

                elif "lithology" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i],
                            color=colors[i], s=sizes[i]
                        )
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], c=colors, s=self.df[element_size].apply(lambda x: map_size(x, 20, 100)))   

                elif "rock unit" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i],
                            marker=self.ds["Shapes"][i], s=sizes[i]
                        )
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], s=self.df[element_size].apply(lambda x: map_size(x, 20, 100)))
                        
                else:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df.iloc[i][axis1], self.df.iloc[i][axis2], self.df.iloc[i][axis3], s=sizes[i],)
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3])

            if "sample id" in self.cleaned_df.columns:
                description = self.cleaned_df["sample id"]
                self.annotations = {}
                self.selected_points = set()
                
                # Functions to handle annotations on the plot
                def add_annotation(ind, source):
                    if source == self.dots:
                        index = ind["ind"][0]
                        pos = source.get_offsets()[index]
                        text = f"{description.iloc[index]}"
                    else:
                        index = ind
                        pos = (self.df.iloc[index][axis1], self.df.iloc[index][axis2])
                        text = f"{description.iloc[index]}"
                
                    offsets = [(20, 20), (-20, 20), (20, -20), (-20, -20), (40, 40), (-40, 40), (40, -40), (-40, -40)]
                    
                    for offset in offsets:
                        annot = ax.annotate(text, xy=pos, xytext=offset,
                                            textcoords="offset points",
                                            bbox=dict(boxstyle="round", fc="w"),
                                            arrowprops=dict(arrowstyle="->"),
                                            zorder=100)
                        fig.canvas.draw()
                        bbox = annot.get_window_extent()
                        overlap = False
                        for existing_annot in self.annotations.values():
                            existing_bbox = existing_annot.get_window_extent()
                            if bbox.overlaps(existing_bbox):
                                overlap = True
                                annot.remove()
                                break
                        if not overlap:
                            annot.set_visible(True)
                            self.annotations[index] = annot
                            self.selected_points.add(index)
                            fig.canvas.draw_idle()
                            return 
                
                    annot = ax.annotate(text, xy=pos, xytext=offsets[0],
                                        textcoords="offset points",
                                        bbox=dict(boxstyle="round", fc="w"),
                                        arrowprops=dict(arrowstyle="->"),
                                        zorder=100)
                    annot.set_visible(True)
                    self.annotations[index] = annot
                    self.selected_points.add(index)
                    fig.canvas.draw_idle()
                
                def remove_annotation(index):
                    annot = self.annotations.pop(index, None)
                    if annot:
                        annot.remove()
                    self.selected_points.discard(index)
                    fig.canvas.draw_idle()
                
                def update_annotations(event):
                    for index, annot in self.annotations.items():
                        if index in self.selected_points:
                            annot.xy = (self.df.iloc[index][axis1], self.df.iloc[index][axis2])
                    fig.canvas.draw_idle()
                
                def on_click(event):
                    if event.inaxes == ax and event.button == MouseButton.LEFT:
                        contains_dots, ind_dots = self.dots.contains(event)
                        contains_shapes = False
                        shape_index = None
                        for i, shape in enumerate(self.shapes):
                            contains_shapes, ind_shapes = shape.contains(event)
                            if contains_shapes:
                                shape_index = i
                                break
                        
                        if contains_dots:
                            index = ind_dots["ind"][0]
                            if index in self.selected_points:
                                remove_annotation(index)
                            else:
                                add_annotation(ind_dots, self.dots)
                        elif contains_shapes:
                            if shape_index in self.selected_points:
                                remove_annotation(shape_index)
                            else:
                                add_annotation(shape_index, shape)
                
                fig.canvas.mpl_connect("button_press_event", on_click)
                fig.canvas.mpl_connect("motion_notify_event", update_annotations)

            # Plot centroids
            scaler = self.pipe.named_steps['scale']

            if self.cluster_result == "BIRCH":
                centroids = self.pipe.named_steps['model'].subcluster_centers_
            elif self.cluster_result == "GMM":
                centroids = self.pipe.named_steps['model'].means_
            elif self.cluster_result == "Spectral" or self.cluster_result == "DBSCAN" or self.cluster_result == "Hierarchical":
                model = self.pipe.named_steps['model']
                labels = model.labels_
                labels = np.array(labels)
                
                unique_labels = np.unique(labels)
                centroids = []
                
                for i in unique_labels:
                    cluster_points = self.pipe.named_steps['scale'].transform(self.X[labels == i])
                    print(f"Cluster {i} points:", cluster_points)
                    centroid = cluster_points.mean(axis=0)
                    centroids.append(centroid)
                        
                centroids = np.array(centroids)

            else:
                centroids = self.pipe.named_steps['model'].cluster_centers_
                
            centroids_original = scaler.inverse_transform(centroids)
            
            ax.scatter(centroids_original[:, 0], centroids_original[:, 1], 
                       marker="X", c="red", s=30, label="centroids")
            ax.legend()
            ax.set_title("K-Means Clustering")
            ax.set_xlabel(axis1)
            ax.set_ylabel(axis2)
            
            canvas = FigureCanvasTkAgg(fig, master=content_frame)
            canvas.draw()

            toolbar = NavigationToolbar2Tk(canvas, content_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)
            
            self.shape()
            Legend = self.var1.get() == 1
            print(Legend)
            if Legend:
                pass
            else:
                self.yellowbrick()
            self.legend()
            
        else:
            self.output_text.insert("end", 'Select two different elements')

    def shape(self):
        # Display or hide shapes in the plot
        shape = self.var.get() == 1
        if shape:
            for shape in self.shapes:
                shape.set_visible(True)
            self.dots.set_visible(False)
        else:
            self.dots.set_visible(True)
            for shape in self.shapes:
                shape.set_visible(False)

    def save_clusters_to_excel(self):
        # Save the clustered data to an Excel file with colored cells based on clusters
        axis1 = self.axis1_combo.get().lower()
        axis2 = self.axis2_combo.get().lower()
        
        if axis1 == axis2:
            print("Select two different elements to save to Excel.")
            return

        if 'cluster' not in self.df.columns:
            print("Clusters have not been calculated yet.")
            return

        if axis1 not in self.cleaned_df.columns or axis2 not in self.cleaned_df.columns or 'sample id' not in self.cleaned_df.columns:
            print(f"One or more of the columns '{axis1}', '{axis2}', or 'sample id' are missing in the data.")
            return

        file_name = fd.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
        if not file_name:
            return

        self.cleaned_df['cluster'] = self.df['cluster']
        
        pivot_data = self.cleaned_df.pivot_table(index='sample id', values=[axis1, axis2, 'cluster'], sort=False)
        
        with pd.ExcelWriter(file_name, engine='openpyxl') as excel_writer:
            pivot_data.to_excel(excel_writer, index=True)
        
        wb = load_workbook(file_name)
        ws = wb.active

        colormap = plt.get_cmap('Set2')
        cluster_colors = ['FF{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255)) for r, g, b, _ in [colormap(i) for i in range(colormap.N)]]
        cluster_fill = [PatternFill(start_color=color, end_color=color, fill_type="solid") for color in cluster_colors]

        for i in range(len(self.cleaned_df)):
            sample_id = self.cleaned_df.iloc[i]['sample id']
            cluster = self.cleaned_df.iloc[i]['cluster']
            row = ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1)
            for cell in row:
                if cell[0].value == sample_id:
                    for col in range(2, ws.max_column + 1):
                        ws.cell(row=cell[0].row, column=col).fill = cluster_fill[cluster]

        wb.save(file_name)
        print(f"Clusters saved to {file_name}")

    def legend(self):
        # Create and display legends for lithology and rock type
        Legend = self.var1.get() == 1
        if Legend:
            print ("yes")
            for widget in self.legend_frame.winfo_children():
                widget.destroy()
            if not self.dc.empty and not self.ds.empty:
                unique_lithologies = self.cleaned_df['lithology'].unique()
                unique_shapes = self.cleaned_df['rock unit'].unique()
        
                # For lithologies
                handle1 = []
                label1 = []
                for lithology in unique_lithologies:
                    color = self.color_map[lithology]
                    l1 = Line2D([], [], color=color, marker='o', linestyle='None', label=lithology)
                    handle1.append(l1)
                    label1.append(lithology)
        
                # For shapes
                handle2 = []
                label2 = []
                for shape in unique_shapes:
                    marker = self.color_map1[shape]
                    l2 = Line2D([], [], color='black', marker=marker, linestyle='None', label=shape)
                    handle2.append(l2)
                    label2.append(shape)
        
                total_entries = len(label1) + len(label2)
                height_per_entry = 0.225
                fig_height = total_entries * height_per_entry
        
                figx = plt.figure(figsize=(2.5, fig_height))
                axx = figx.add_subplot(111)
                axx.axis('off')
        
                self.legend1 = plt.legend(handle1, label1, bbox_to_anchor=(0.9, 1.15), title='Lithology', fontsize=10, labelspacing=0.3)
                self.legend2 = plt.legend(handle2, label2, bbox_to_anchor=(0.9, 0.35), title='Rock Type', fontsize=10, labelspacing=0.3)
                axx.add_artist(self.legend1)
                axx.add_artist(self.legend2)
                self.canvas1 = FigureCanvasTkAgg(figx, master=self.legend_frame)
                toolbar = CustomToolbar(self.canvas1, self.legend_frame)
                toolbar.update()
                toolbar.pack(side=tk.TOP, fill=tk.X)
        
                self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        
                self.legend1.set_visible(self.var1.get() == 1)
                self.legend2.set_visible(self.var1.get() == 1)
            else:
                pass
        else:
            pass







class Cluster3DPlotClass:
    def __init__(self, shared_container, cluster, df, cleaned_df, box_frame, box_frame_sub, dc, ds, color_map, color_map1, on_button_click, legend_frame):
        # Initialize class variables
        self.df = df.dropna()
        self.shared_container = shared_container
        self.cluster_result = cluster
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.legend_frame = legend_frame
        self.cleaned_df = cleaned_df
        self.on_button_click = on_button_click

        self.dc = dc
        self.ds = ds
        self.color_map = color_map
        self.color_map1 = color_map1
        self.var = IntVar()
        self.var1 = IntVar()

        self.plot_3d_cluster()

    def plot_3d_cluster(self):
        # Load and resize the image for the 3D plot button
        pil_image = Image.open("images program/3d.png")
        self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))
        
        # Create a label with the icon and bind a click event to it
        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text="", width=20)
        self.image_button.grid(row=3, column=2, sticky="w", pady=0, padx=5)
        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, lambda: self.plot_3d_cluster_sub(self.cluster_result)))

        # Add a tooltip to the label
        ToolTip(self.image_button, msg="PCA 3D Biplot with Clusters")
        
    def plot_3d_cluster_sub(self, cluster):
        self.cluster_result = cluster
        print("done")
        print(self.cluster_result)
        # Clear any existing widgets in the sub-frame
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()
        print("donedone")
        # Create UI elements for selecting axes and other options
        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 1:").grid(row=0, column=0, pady=(5,0), padx=5)
        self.axis1_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis1_combo.current(0)
        self.axis1_combo.grid(row=1, column=0, pady=(3,0), padx=5)
        
        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 2:").grid(row=2, column=0, pady=(5,0), padx=5)
        self.axis2_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis2_combo.current(2)
        self.axis2_combo.grid(row=3, column=0, pady=(3,0), padx=5)

        ctk.CTkLabel(self.box_frame_sub, text="Select Axis 3:").grid(row=4, column=0, pady=(5,0), padx=5)
        self.axis3_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist())
        self.axis3_combo.current(4)
        self.axis3_combo.grid(row=5, column=0, pady=(3,0), padx=5)
        
        #select element for data point size
        ctk.CTkLabel(self.box_frame_sub, text="Select Element for Data Point Size").grid(row=6, column=0, pady=(5,0), padx=5)
        self.size_combo = ttk.Combobox(self.box_frame_sub, values=self.df.columns.tolist(), name="size_combo")
        self.size_combo.set("N/A")
        self.size_combo.grid(row=7, column=0, pady=(3,5), padx=5)
        
        # Add checkbox for displaying shapes if ds is not empty
        if not self.ds.empty:
            self.checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Display shape", variable=self.var)
            self.checkbox.grid(row=8, column=0, sticky=W, pady=(5,0), padx=5)

        self.legend_checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Show Legend", variable=self.var1)
        self.legend_checkbox.grid(row=9, column=0, sticky=W, pady=(5,0), padx=5)
        
        # Add checkbox for coloring by cluster if dc is not empty
        if not self.dc.empty:
            self.color_checkbox_var = IntVar()
            self.color_checkbox = ctk.CTkCheckBox(self.box_frame_sub, text="Color by Cluster", variable=self.color_checkbox_var)
            self.color_checkbox.grid(row=10, column=0, sticky=W, pady=(5,0), padx=5)
        
        # Add slider for selecting the number of clusters
        self.cluster_text = ctk.CTkLabel(self.box_frame_sub, text="Number of Clusters:")
        self.k_slider = tk.Scale(self.box_frame_sub, from_=2, to=10, orient=tk.HORIZONTAL)

        if self.cluster_result in ["DBSCAN", "Mean Shift", "Affinity Propagation"]:
            self.cluster_text.grid_forget()
            self.k_slider.grid_forget()
        else:
            self.cluster_text.grid(row=11, column=0, pady=(5,0), padx=5)
            self.k_slider.grid(row=12, column=0, pady=(3,0), padx=5)
        
        # Add apply button
        self.apply_plot_3d_cluster = ctk.CTkButton(self.box_frame_sub, text="Apply", command=self.show_cluster)
        self.apply_plot_3d_cluster.grid(row=13, column=0, pady=(5,0), padx=5)

        # Add save button
        self.save_button = ctk.CTkButton(self.box_frame_sub, text="Save Clusters to Excel", command=self.save_clusters_to_excel)
        self.save_button.pack(side="bottom", padx=5, pady=5)

    def yellowbrick(self): 
        # Use Yellowbrick's KElbowVisualizer to help determine the optimal number of clusters
        try:
            plt.rcParams.update({'font.size': 10})
            
            fig = plt.figure(figsize=(3, 2))
            ax = fig.add_subplot(111)
            ax.axis('off')
            
            model = self.pipe.named_steps['model']
            if 'n_clusters' in model.get_params():
                del model.get_params()['n_clusters']
                
            visualizer = KElbowVisualizer(model, k=(1, 12), timing=False, title="KElbowVisualizer").fit(self.X)
            
            visualizer.ax.set_yticklabels([])
            visualizer.finalize()
            
            self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
            toolbar = CustomToolbar(self.canvas1, self.legend_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)
            self.canvas1.get_tk_widget().pack(fill="both", expand=True)
            
            plt.close(fig)
        except:
            pass

    def estimate_eps(self, k):
        # Estimate the epsilon value for DBSCAN using the k-nearest neighbors method
        from sklearn.neighbors import NearestNeighbors
        for widget in self.legend_frame.winfo_children():
            widget.destroy()

        # Compute the k-nearest neighbors
        nearest_neighbors = NearestNeighbors(n_neighbors=k)
        nearest_neighbors.fit(self.X)
        distances, indices = nearest_neighbors.kneighbors(self.X)

        # Sort the distances (distances are ordered for each point)
        distances = np.sort(distances, axis=0)
        distances = distances[:, 1]  # Taking the distance to the k-th nearest neighbor

        # Plot the k-distance graph
        fig, ax = plt.subplots(figsize=(3, 2))
        ax.plot(distances)
        ax.set_title('k-Distance Graph')
        ax.set_xlabel('Points sorted by distance')
        ax.set_ylabel(f'{k}th Nearest Neighbor Distance')

        # Display the plot in the legend frame
        self.canvas1 = FigureCanvasTkAgg(fig, master=self.legend_frame)
        self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        plt.show()

        # Estimate epsilon from the k-distance graph
        elbow_point_index = 80  
        eps_value = distances[elbow_point_index]
        return eps_value
        
    def show_cluster(self):
        # Display the 3D cluster plot based on the selected axes and clustering algorithm
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("3D Cluster")

        plt.close('all')
        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()
        for widget in self.legend_frame.winfo_children():
            widget.destroy()
            
        axis1 = self.axis1_combo.get()
        axis2 = self.axis2_combo.get()
        axis3 = self.axis3_combo.get()
        k = self.k_slider.get()

        # Ensure the selected axes are different
        if axis1 != axis2 and axis1 != axis3 and axis2 != axis3:
            # Clean and prepare data for clustering
            for col in [axis1, axis2, axis3]:
                self.df[col] = self.df[col].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
            self.df = self.df.astype({axis1: 'float', axis2: 'float', axis3: 'float'})

            self.X = self.df[[axis1, axis2, axis3]].values
            
            # Remove previous cluster result label if it exists
            if hasattr(self, 'cluster_print') and self.cluster_print is not None:
                self.cluster_print.destroy()
                del self.cluster_print

            # Get the selected clustering algorithm
            cluster_result = self.cluster_result
            self.cluster_print = ctk.CTkLabel(self.box_frame_sub, text=f"Cluster: {cluster_result}")
            self.cluster_print.grid(row=14, column=0, pady=(5,0), padx=5)
            
            # Set up the clustering pipeline based on the selected algorithm
            if cluster_result == "K-mean":
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", KMeans(n_clusters=k, random_state=0, n_init='auto'))
                ])
                self.cluster_text.grid(row=11, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=12, column=0, pady=(3,0), padx=5)
                
            elif cluster_result == "DBSCAN":
                min_samples_value = 10
                eps_value = self.estimate_eps(min_samples_value)
                
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", DBSCAN(eps=eps_value, min_samples=min_samples_value))
                ])
                self.cluster_text.grid_forget()
                self.k_slider.grid_forget()
                
            elif cluster_result == "Mean Shift":  
                from sklearn.cluster import estimate_bandwidth
                bandwidth_value = estimate_bandwidth(self.X, quantile=0.2)
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", MeanShift(bandwidth=bandwidth_value))
                ])
                self.cluster_text.grid_forget()
                self.k_slider.grid_forget()
                    
            elif cluster_result == "Spectral":  
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=0))
                ])
                self.cluster_text.grid(row=11, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=12, column=0, pady=(3,0), padx=5)
                
            elif cluster_result == "GMM":   
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", GaussianMixture(n_components=k, random_state=0))
                ])
                self.cluster_text.grid(row=11, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=12, column=0, pady=(3,0), padx=5)
                
            elif cluster_result == "Affinity Propagation":   
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", AffinityPropagation(random_state=0))
                ])
                self.cluster_text.grid_forget()
                self.k_slider.grid_forget()
                
            elif cluster_result == "Hierarchical":
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", AgglomerativeClustering(n_clusters=k, linkage='ward'))
                ])
                self.cluster_text.grid(row=11, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=12, column=0, pady=(3,0), padx=5)
                
            elif cluster_result == "BIRCH":
                self.pipe = Pipeline([
                    ("scale", StandardScaler()),
                    ("model", Birch(n_clusters=k))
                ])
                self.cluster_text.grid(row=11, column=0, pady=(5,0), padx=5)
                self.k_slider.grid(row=12, column=0, pady=(3,0), padx=5)
    
            # Fit the clustering pipeline to the data
            self.pipe.fit(self.X)
            
            # Get cluster labels
            if cluster_result == "GMM":  
                self.df['cluster'] = self.pipe.named_steps["model"].predict(self.X)
            else:
                self.df['cluster'] = self.pipe.named_steps['model'].labels_

            # Create a 3D plot
            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, projection='3d')
                
            self.shapes = []

            # Color points by cluster if the checkbox is selected
            color_by_cluster = self.color_checkbox_var.get() == 1
            if color_by_cluster:
                unique_clusters = self.df['cluster'].unique()
                colormap = plt.get_cmap('tab10')
                cluster_colors = {cluster: colormap(i / len(unique_clusters)) for i, cluster in enumerate(unique_clusters)}
                colors = self.df['cluster'].map(cluster_colors)
            else:
                colors = self.dc['Color']
            
            # Plot the points with shapes and colors
            element_size = self.size_combo.get()
            if element_size == "N/A":
                if "lithology" in self.cleaned_df.columns and "rock unit" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i], self.df[axis3][i],
                            color=colors[i], marker=self.ds["Shapes"][i]
                        )
                        self.shapes.append(shape)

                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3], c=colors)

                elif "lithology" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i], self.df[axis3][i],
                            color=colors[i]
                        )
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3], c=colors)                        

                elif "rock unit" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i], self.df[axis3][i],
                            marker=self.ds["Shapes"][i]
                        )
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3])                        

                else:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df.iloc[i][axis1], self.df.iloc[i][axis2], self.df.iloc[i][axis3])
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3])
            else:
                # Try to get element_size column values for size mapping
                self.max_ele = self.df[element_size].max()
                self.min_ele = self.df[element_size].min()
                
                # Function to map element_size values to point sizes
                def map_size(x, new_min, new_max):
                    old_min = self.min_ele
                    old_max = self.max_ele
                    return ((np.log(x + 1) - np.log(old_min + 1)) / (np.log(old_max + 1) - np.log(old_min + 1))) * (new_max - new_min) + new_min
                    
                sizes = self.df[element_size].apply(lambda x: map_size(x, 20, 100))

                if "lithology" in self.cleaned_df.columns and "rock unit" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i], self.df[axis3][i],
                            color=colors[i], marker=self.ds["Shapes"][i], s=sizes[i]
                        )
                        self.shapes.append(shape)   
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3], c=colors, s=self.df[element_size].apply(lambda x: map_size(x, 20, 100)))

                elif "lithology" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i], self.df[axis3][i],
                            color=colors[i], s=sizes[i]
                        )
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3], c=colors, s=self.df[element_size].apply(lambda x: map_size(x, 20, 100)))   

                elif "rock unit" in self.cleaned_df.columns:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df[axis1][i], self.df[axis2][i], self.df[axis3][i],
                            marker=self.ds["Shapes"][i], s=sizes[i]
                        )
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3], s=self.df[element_size].apply(lambda x: map_size(x, 20, 100)))
                        
                else:
                    for i in range(len(self.df)):
                        shape = ax.scatter(self.df.iloc[i][axis1], self.df.iloc[i][axis2], self.df.iloc[i][axis3], s=sizes[i],)
                        self.shapes.append(shape)
                    self.dots = ax.scatter(self.df[axis1], self.df[axis2], self.df[axis3])


            # Annotate points with sample IDs if available
            if "sample id" in self.cleaned_df.columns:
                description = self.cleaned_df["sample id"]
                self.annotations = {}
                self.selected_points = set()
            
                from mpl_toolkits.mplot3d import proj3d
                
                def add_annotation(ind, source):
                    if source == self.dots:
                        index = ind["ind"][0]
                        pos = source.get_offsets()[index]
                        text = f"{description.iloc[index]}"
                    else:
                        index = ind
                        x, y, _ = proj3d.proj_transform(self.df.iloc[index][axis1], self.df.iloc[index][axis2], self.df.iloc[index][axis3], ax.get_proj())
                        pos = (x, y)
                        text = f"{description.iloc[index]}"

                    offsets = [(20, 20), (-20, 20), (20, -20), (-20, -20), (40, 40), (-40, 40), (40, -40), (-40, -40)]
                    
                    for offset in offsets:
                        annot = ax.annotate(text, xy=pos, xytext=offset,
                                            textcoords="offset points",
                                            bbox=dict(boxstyle="round", fc="w"),
                                            arrowprops=dict(arrowstyle="->",),
                                            zorder=100)
                        fig.canvas.draw()
                        bbox = annot.get_window_extent()
                        overlap = False
                        for existing_annot in self.annotations.values():
                            existing_bbox = existing_annot.get_window_extent()
                            if bbox.overlaps(existing_bbox):
                                overlap = True
                                annot.remove()
                                break
                        if not overlap:
                            annot.set_visible(True)
                            self.annotations[index] = annot
                            self.selected_points.add(index)
                            fig.canvas.draw_idle()
                            return 
                
                    annot = ax.annotate(text, xy=pos, xytext=offsets[0],
                                        textcoords="offset points",
                                        bbox=dict(boxstyle="round", fc="w"),
                                        arrowprops=dict(arrowstyle="->"),
                                        zorder=100)
                    annot.set_visible(True)
                    self.annotations[index] = annot
                    self.selected_points.add(index)
                    fig.canvas.draw_idle()
                
                def remove_annotation(index):
                    annot = self.annotations.pop(index, None)
                    if annot:
                        annot.remove()
                    self.selected_points.discard(index)
                    fig.canvas.draw_idle()
                
                def update_annotations(event):
                    for index, annot in self.annotations.items():
                        if index in self.selected_points:
                            x, y, _ = proj3d.proj_transform(self.df.iloc[index][axis1], self.df.iloc[index][axis2], self.df.iloc[index][axis3], ax.get_proj())
                            annot.xy = (x, y)
                    fig.canvas.draw_idle()
                
                def on_click(event):
                    if event.inaxes == ax and event.button == MouseButton.LEFT:
                        contains_dots, ind_dots = self.dots.contains(event)
                        contains_shapes = False
                        shape_index = None
                        for i, shape in enumerate(self.shapes):
                            contains_shapes, ind_shapes = shape.contains(event)
                            if contains_shapes:
                                shape_index = i
                                break
                        
                        if contains_dots:
                            index = ind_dots["ind"][0]
                            if index in self.selected_points:
                                remove_annotation(index)
                            else:
                                add_annotation(ind_dots, self.dots)
                        elif contains_shapes:
                            if shape_index in self.selected_points:
                                remove_annotation(shape_index)
                            else:
                                add_annotation(shape_index, shape)
                
                fig.canvas.mpl_connect("button_press_event", on_click)
                fig.canvas.mpl_connect("motion_notify_event", update_annotations)

            # Get centroids of the clusters
            scaler = self.pipe.named_steps['scale']
            if cluster_result == "BIRCH":
                centroids = self.pipe.named_steps['model'].subcluster_centers_
            elif cluster_result == "GMM":
                centroids = self.pipe.named_steps['model'].means_
            elif cluster_result == "Spectral" or cluster_result == "DBSCAN" or cluster_result == "Hierarchical":
                model = self.pipe.named_steps['model']
                labels = model.labels_
                labels = np.array(labels)
                
                unique_labels = np.unique(labels)
                centroids = []
                
                for i in unique_labels:
                    cluster_points = self.pipe.named_steps['scale'].transform(self.X[labels == i])
                    centroid = cluster_points.mean(axis=0)
                    centroids.append(centroid)
                        
                centroids = np.array(centroids)
            else:
                centroids = self.pipe.named_steps['model'].cluster_centers_
                
            # Transform centroids back to original scale
            centroids_original = scaler.inverse_transform(centroids)
            
            # Plot centroids in red with X marker
            ax.scatter(centroids_original[:, 0], centroids_original[:, 1], centroids_original[:, 2], 
                       marker="X", c="red", s=100, label="centroids")
        
            ax.set_xlabel(axis1)
            ax.set_ylabel(axis2)
            ax.set_zlabel(axis3)
            plt.legend()
            plt.title("3D K-Means Clustering")

            # Create a canvas for the plot and add toolbar
            canvas = FigureCanvasTkAgg(fig, master=content_frame)
            canvas.draw()
            toolbar = NavigationToolbar2Tk(canvas, content_frame)
            toolbar.update()
            toolbar.pack(side=tk.TOP, fill=tk.X)
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)

            # Toggle shape visibility and call Yellowbrick visualizer
            self.shape()
            Legend = self.var1.get() == 1
            print(Legend)
            if Legend:
                pass
            else:
                self.yellowbrick()
            self.legend()
            
        else:
            self.output_text.insert("end", 'Select three different elements')

    def shape(self):
        # Toggle the visibility of shapes in the plot
        shape = self.var.get() == 1
        if shape:
            for shape in self.shapes:
                shape.set_visible(True)
            self.dots.set_visible(False)
        else:
            self.dots.set_visible(True)
            for shape in self.shapes:
                shape.set_visible(False)

    def save_clusters_to_excel(self):
        # Save the clustered data to an Excel file with colored cells based on clusters
        axis1 = self.axis1_combo.get().lower()
        axis2 = self.axis2_combo.get().lower()
        axis3 = self.axis3_combo.get().lower()
        
        # Ensure that the selected axes are different
        if axis1 == axis2 == axis3:
            print("Select three different elements to save to Excel.")
            return

        if 'cluster' not in self.df.columns:
            print("Clusters have not been calculated yet.")
            return

        # Ensure the required columns exist in the cleaned DataFrame
        if axis1 not in self.cleaned_df.columns or axis2 not in self.cleaned_df.columns or axis3 not in self.cleaned_df.columns or 'sample id' not in self.cleaned_df.columns:
            print(f"One or more of the columns '{axis1}', '{axis2}', '{axis3}', or 'sample id' are missing in the data.")
            returnv

        # Prompt the user to select a file name for saving the Excel file
        file_name = fd.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
        if not file_name:
            return
        
        # Add the cluster labels to the cleaned DataFrame
        self.cleaned_df['cluster'] = self.df['cluster']
        
        # Pivot the data to have sample IDs as index and selected elements as columns
        pivot_data = self.cleaned_df.pivot_table(index='sample id', values=[axis1, axis2, axis3, 'cluster'], sort=False)
        
        # Save the pivoted data to an Excel file
        with pd.ExcelWriter(file_name, engine='openpyxl') as excel_writer:
            pivot_data.to_excel(excel_writer, index=True)
        
        wb = load_workbook(file_name)
        ws = wb.active

        # Generate colors for each cluster
        colormap = plt.get_cmap('Set2')
        cluster_colors = ['FF{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255)) for r, g, b, _ in [colormap(i) for i in range(colormap.N)]]
        cluster_fill = [PatternFill(start_color=color, end_color=color, fill_type="solid") for color in cluster_colors]

        # Apply color to each cell based on cluster
        for i in range(len(self.cleaned_df)):
            sample_id = self.cleaned_df.iloc[i]['sample id']
            cluster = self.cleaned_df.iloc[i]['cluster']
            row = ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1)
            for cell in row:
                if cell[0].value == sample_id:
                    for col in range(2, ws.max_column + 1):
                        ws.cell(row=cell[0].row, column=col).fill = cluster_fill[cluster]

        wb.save(file_name)
        print(f"Clusters saved to {file_name}")

    def legend(self):
        # Create and display legends for lithology and rock type
        Legend = self.var1.get() == 1
        if Legend:
            print ("yes")
            for widget in self.legend_frame.winfo_children():
                widget.destroy()
            if not self.dc.empty and not self.ds.empty:
                unique_lithologies = self.cleaned_df['lithology'].unique()
                unique_shapes = self.cleaned_df['rock unit'].unique()
        
                # For lithologies
                handle1 = []
                label1 = []
                for lithology in unique_lithologies:
                    color = self.color_map[lithology]
                    l1 = Line2D([], [], color=color, marker='o', linestyle='None', label=lithology)
                    handle1.append(l1)
                    label1.append(lithology)
        
                # For shapes
                handle2 = []
                label2 = []
                for shape in unique_shapes:
                    marker = self.color_map1[shape]
                    l2 = Line2D([], [], color='black', marker=marker, linestyle='None', label=shape)
                    handle2.append(l2)
                    label2.append(shape)
        
                total_entries = len(label1) + len(label2)
                height_per_entry = 0.225
                fig_height = total_entries * height_per_entry
        
                figx = plt.figure(figsize=(2.5, fig_height))
                axx = figx.add_subplot(111)
                axx.axis('off')
        
                self.legend1 = plt.legend(handle1, label1, bbox_to_anchor=(0.9, 1.15), title='Lithology', fontsize=10, labelspacing=0.3)
                self.legend2 = plt.legend(handle2, label2, bbox_to_anchor=(0.9, 0.35), title='Rock Type', fontsize=10, labelspacing=0.3)
                axx.add_artist(self.legend1)
                axx.add_artist(self.legend2)
                self.canvas1 = FigureCanvasTkAgg(figx, master=self.legend_frame)
                toolbar = CustomToolbar(self.canvas1, self.legend_frame)
                toolbar.update()
                toolbar.pack(side=tk.TOP, fill=tk.X)
        
                self.canvas1.get_tk_widget().pack(fill="both", expand=True)
        
                self.legend1.set_visible(self.var1.get() == 1)
                self.legend2.set_visible(self.var1.get() == 1)
            else:
                pass
        else:
            pass






class drill_class:
    def __init__(self, shared_container, pca_df_scaled, cleaned_df, df, box_frame, box_frame_sub, on_button_click, apply_button):
        self.shared_container = shared_container
        self.apply_button = apply_button
        self.on_button_click = on_button_click
        self.pca_df_scaled = pca_df_scaled
        self.cleaned_df = cleaned_df
        self.df = df
        self.box_frame = box_frame
        self.box_frame_sub = box_frame_sub
        self.var = IntVar()

        # Initialize the drill hole graph
        self.Graph_PCA()

    def Graph_PCA(self):
        # Load and resize the image for the button icon
        pil_image = Image.open("images program/drill-svgrepo-com.png")
        self.icon_image = CTkImage(light_image=pil_image, dark_image=pil_image, size=(32, 32))

        # Create a label with the icon and bind a click event to it
        self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text="", width=20)
        self.image_button.grid(row=1, column=4, sticky="w", pady=0, padx=5)
        self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, self.Graph_PCA_sub))

        # Add a tooltip to the label
        ToolTip(self.image_button, msg="Drill Hole Chart")

    def Graph_PCA_sub(self):
        # Clean and process sample IDs
        self.cleaned_df['Sample_ID_Clean'] = self.cleaned_df['sample id'].apply(lambda x: x.replace('-', ''))
        self.cleaned_df['Drill_Hole'] = self.cleaned_df['Sample_ID_Clean'].apply(lambda x: x[:3])
        
        # Extract depth unit from column name
        column_name = self.cleaned_df.filter(like='depth').columns[0]
        match = re.search(r'\((.*?)\)', column_name)
        self.unit = match.group(1) if match else None
        
        self.cleaned_df['Depth'] = self.cleaned_df[column_name]
        self.drill_holes = sorted(self.cleaned_df['Drill_Hole'].unique().tolist())

        # Clear previous widgets
        for widget in self.box_frame_sub.winfo_children():
            widget.destroy()

        # Create a dropdown for selecting drill hole
        ttk.Label(self.box_frame_sub, text="Select Drill Hole:").pack(side="top", padx=5, pady=(5, 0))
        self.drill_hole_combo = ttk.Combobox(self.box_frame_sub, values=self.drill_holes)
        self.drill_hole_combo.current(0)
        self.drill_hole_combo.pack(side="top", padx=5, pady=(5, 0))

        # Add an apply button
        self.apply_Graph_PCA = ctk.CTkButton(self.box_frame_sub, text="apply", command=self.update_plots)
        self.apply_Graph_PCA.pack(side="top", padx=5, pady=5)

    def normalize(self, values, vmin=None, vmax=None):
        # Normalize values to range [0, 1]
        vmin = vmin if vmin is not None else np.min(values)
        vmax = vmax if vmin is not None else np.max(values)
        norm_values = (values - vmin) / (vmax - vmin)
        return norm_values

    def plot_pca_scatter(self, data, y_positions, ax):
        # Normalize data and create scatter plot with color mapping
        norm_values = self.normalize(data, vmin=-max(abs(data)), vmax=max(abs(data)))
        scatter_colors = plt.cm.coolwarm(norm_values)
        ax.scatter(data, y_positions, color=scatter_colors, s=30)
        ax.set_yticks(y_positions)
        ax.set_yticklabels([])
        ax.invert_yaxis()
        ax.grid(True, linestyle='--', linewidth=0.5)
        ax.axvline(x=0, color='black', linewidth=0.5)
        max_val = max(abs(data.min()), data.max())
        ax.set_xlim(-max_val, max_val)
        ax.tick_params(rotation=-35, labelsize="medium")

        # Customize plot spines
        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1)

    def plot_depth_chart(self, depths, y_positions, ax):
        # Plot depth chart with horizontal lines representing depths
        ax.hlines(y=y_positions, xmin=0, xmax=0.2, linestyles='-', linewidth=0.8)
        ax.set_yticks(y_positions)
        ax.set_yticklabels(depths)
        ax.invert_yaxis()
        ax.set_xticklabels([])
        ax.set_title(f'Depth ({self.unit})')
        ax.axis('on')
        ax.set_ylabel(self.drill_hole)
        ax.tick_params(labelsize="small")

    def update_plots(self):
        # Create a new tab if not already present
        if not self.shared_container.current_tab:
            self.shared_container.create_tab("Drill Hole")

        content_frame = self.shared_container.current_tab[1]
        for widget in content_frame.winfo_children():
            widget.destroy()
        
        # Filter and sort data for the selected drill hole
        self.drill_hole = self.drill_hole_combo.get()
        filtered_cleaned_df = self.cleaned_df[self.cleaned_df['Drill_Hole'] == self.drill_hole]
        if filtered_cleaned_df.empty:
            self.output_text.insert("end", "No samples found for the selected drill hole.")
            return

        filtered_cleaned_df = filtered_cleaned_df.sort_values(by='Depth')
        sample_indices = filtered_cleaned_df.index
        depths = filtered_cleaned_df['Depth'].tolist()
        if len(sample_indices) == 0:
            self.output_text.insert("end", "No samples found for the selected drill hole.")
            return

        # Calculate y positions for depth markers
        print(depths)
        max_depth = max(depths)
        min_depth = min(depths)
        if max_depth == min_depth:
            y_positions = [0.5 for _ in depths]
        else:
            y_positions = [((depth - min_depth) / (max_depth - min_depth)) * len(depths) for depth in depths]

        plt.close('all')

        # Create subplots for depth chart and PCA scatter plots
        fig, axes = plt.subplots(nrows=1, ncols=len(self.pca_df_scaled.columns) + 1, figsize=(15, 10))
        pos1 = axes[0].get_position()
        axes[0].set_position([pos1.x0, pos1.y0, pos1.width * 0.5, pos1.height])
        self.plot_depth_chart(depths, y_positions, axes[0])

        # Plot PCA scatter plots for each principal component
        for col, pc in enumerate(self.pca_df_scaled.columns):
            data = self.pca_df_scaled.loc[sample_indices, pc]
            ax = axes[col + 1]
            self.plot_pca_scatter(data, y_positions, ax)
            ax.set_title(f'PC{col + 1}')

        # Embed the plot in the Tkinter GUI
        canvas = FigureCanvasTkAgg(fig, master=content_frame)
        canvas.draw()
        toolbar = NavigationToolbar2Tk(canvas, content_frame)
        toolbar.update()
        toolbar.pack(side=tk.TOP, fill=tk.X)
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True, in_=content_frame)


    # def Graph_PCA(self):
    #     pil_image = Image.open("images program/drill-svgrepo-com.png")
    #     #resized_image = pil_image.resize((32, 32), Image.LANCZOS)
    #     self.icon_image = ctk.CTkImage(light_image=pil_image, size=(32, 32))
    #     #self.icon_image = ImageTk.PhotoImage(resized_image)
        
    #     self.image_button = ctk.CTkLabel(self.box_frame, image=self.icon_image, text = "", width=20)
    #     self.image_button.grid(row=4, column=0, sticky="w", pady=(5,0), padx=(5,0))

    #     self.image_button.bind("<Button-1>", lambda event: self.on_button_click(self.image_button, self.Graph_PCA_sub))

    #     ToolTip(self.image_button, msg="Drill Hole Chart")
             
    #     # #self.graph_bar_button = ctk.CTkButton(self.box_frame, text="Drill PC graph", command=lambda: self.on_button_click(self.graph_bar_button, self.Graph_PCA_sub))

        
    # #     self.icon_image = Image.open("images program/chart-scatter-3d-svgrepo-com.png")
    # #     print(self.icon_image)
    # #     self.ctk_icon_image = ctk.CTkImage(light_image=self.icon_image, size=(24, 24))
    # #     print(self.ctk_icon_image)
        
    # #     self.graph_bar_button = ctk.CTkButton(
    # #         self.box_frame, 
    # #         image=self.ctk_icon_image, 
    # #         text="",
    # #         command=lambda: self.on_button_click(self.graph_bar_button, self.Graph_PCA_sub)
    # #     )
      
    # #     self.graph_bar_button.pack(side="top", padx=5, pady=(5, 0))







class CustomToolbar(NavigationToolbar2Tk):
    #Toolbar with only the save button
    def __init__(self, canvas, parent):
        super().__init__(canvas, parent)

        for widget in self.winfo_children():
            try:
                if isinstance(widget, ttk.Separator):
                    widget.pack_forget()
                elif str(widget['text']) != 'Save':
                    widget.pack_forget()
            except (tk.TclError, KeyError):
                pass



if __name__ == "__main__":
    app = main()
    app.mainloop()






2024-07-31 14:38:01.247 python[78282:2040349] +[CATransaction synchronize] called within transaction
2024-07-31 14:38:02.976 python[78282:2040349] +[CATransaction synchronize] called within transaction


<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=32x32 at 0x1042890D0>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=32x32 at 0x16A813150>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=32x32 at 0x16A85D310>
pyimage6


/opt/anaconda3/lib/python3.11/site-packages/customtkinter/windows/widgets/core_widget_classes/ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


done
K-mean
donedone
done
K-mean
donedone
done
K-mean
donedone


In [ ]:
#prediction
# from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline
# try:
#     y = dc['Lithology']
#     y = y.values.ravel()
#     X = pca_df_scaled
#     print("Shape of y:", y.shape)
#     print("Shape of X:", X.shape)
    
#     clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
#     clf.fit(X, y)
    
#     print(clf.predict([[0.3, -0.01, 0.1]]))
# except:
#     print ("need more litho group")

# memory_4 = get_memory_usage()
# print(f"Memory Usage After predict: {memory_4:.2f} MB")

In [ ]:
            #3d background shading, use only if enough memory (mac impossible)
            # x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
            # y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
            # z_min, z_max = X[:, 2].min() - 1, X[:, 2].max() + 1
            # xx, yy, zz = np.meshgrid(np.linspace(x_min, x_max, 50),
            #                          np.linspace(y_min, y_max, 50),
            #                          np.linspace(z_min, z_max, 50))
        
            # # Predict cluster labels for each point in the mesh grid
            # grid_points = np.c_[xx.ravel(), yy.ravel(), zz.ravel()]
            # grid_points_scaled = pipe.named_steps['scale'].transform(grid_points)
            # Z = pipe.named_steps['model'].predict(grid_points_scaled)
            # Z = Z.reshape(xx.shape)
        
            # ax.scatter(xx, yy, zz, c=Z, cmap='Pastel1', alpha=0.1)

In [ ]:
#how to use nagivation bar, how to same best quality figure, how to set up excel